# Vajalike teekide installimine

In [3]:
!python --version

Python 3.7.10


In [4]:
# Installi Estnltk 1.6(7b0)
!pip install conllu==3.1.1 estnltk==1.6.7b0


     |████████████████████████████████| 99.4MB 54kB/s 
     |████████████████████████████████| 747kB 36.0MB/s 
     |████████████████████████████████| 1.5MB 30.4MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# Imports ja *file paths*

In [5]:
# Imports
import estnltk
import sys, os, re
from estnltk.converters import text_to_json, json_to_text
import json
import pickle
# impordime ajaväljendite tuvastaja Resolver-i
from estnltk.taggers.standard_taggers.timex_tagger_preprocessing import TIMEXES_RESOLVER

drive_path = "Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/"
baseAnnotationFile     = drive_path + "EstTimeMLCorpus/corpus/base-segmentation-morph-syntax"
eventAnnotationFile    = drive_path + "EstTimeMLCorpus/corpus/event-annotation"
timexAnnotationFile    = drive_path + "EstTimeMLCorpus/corpus/timex-annotation"
timexAnnotationDCTFile = drive_path + "EstTimeMLCorpus/corpus/timex-annotation-dct"
tlinkEventTimexFile    = drive_path + "EstTimeMLCorpus/corpus/tlink-event-timex"
tlinkEventDCTFile      = drive_path + "EstTimeMLCorpus/corpus/tlink-event-dct"
tlinkMainEventsFile    = drive_path + "EstTimeMLCorpus/corpus/tlink-main-events"
tlinkSubEventsFile     = drive_path + "EstTimeMLCorpus/corpus/tlink-subordinate-events"
articleMetadata_file   = drive_path + "EstTimeMLCorpus/corpus/article-metadata"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Vajalik Colabist korpuse lugemiseks

See Jupyteri fail asub kaustas, kus asub EstTimeMLCorpus korpuse kaust.

In [6]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [ ]:
# Korpuse paremini mõistmiseks on loodud kood. Eemalda kommentaar ja jooksuta.

# !python EstTimeMLCorpus/exported_corpus_reader.py EstTimeMLCorpus/corpus

# Meetodid korpuses olevate andmete lugemiseks. (viidata korpusele, kood on kopeeritud)

In [7]:
def load_base_segmentation(inputFile): # VIIDATA
    base_segmentation = dict()
    last_sentenceID = ""
    f = open(inputFile, mode='r', encoding="utf-8")
    for line in f:
        # Skip the comment line
        if ( re.match("^#.+$", line) ):
            continue
        items = (line.rstrip()).split("\t")
        if (len(items) != 7):
            raise Exception(" Unexpected number of items on line: '"+str(line)+"'")
        file = items[0]
        if (file not in base_segmentation):
            base_segmentation[file] = []
        sentenceID = items[1]
        if (sentenceID != last_sentenceID):
            base_segmentation[file].append([])
        wordID     = items[2]
        # fileName	sentence_ID	word_ID_in_sentence	token	morphological_and_syntactic_annotations	syntactic_ID	syntactic_ID_of_head
        token           = items[3]
        morphSyntactic  = items[4]
        syntacticID     = items[5]
        syntacticHeadID = items[6]
        base_segmentation[file][-1].append( [sentenceID, wordID, token, morphSyntactic, syntacticID, syntacticHeadID] )
        last_sentenceID = sentenceID
    f.close()
    return base_segmentation

def load_entity_annotation(inputFile): # VIIDATA
    annotationsByLoc = dict()
    annotationsByID  = dict()
    f = open(inputFile, mode='r', encoding="utf-8")
    for line in f:
        # Skip the comment line
        if ( re.match("^#.+$", line) ):
            continue
        items = (line.rstrip()).split("\t")
        # fileName	sentence_ID	word_ID_in_sentence	expression	event_annotation	event_ID
        # fileName	sentence_ID	word_ID_in_sentence	expression	timex_annotation	timex_ID
        if (len(items) != 6):
            raise Exception(" Unexpected number of items on line: '"+str(line)+"'")
        file       = items[0]
        sentenceID = items[1]
        wordID     = items[2]
        expression = items[3]
        annotation = items[4]
        entityID   = items[5]
        if (file not in annotationsByLoc):
            annotationsByLoc[file] = dict()
        if (file not in annotationsByID):
            annotationsByID[file] = dict()
        # Record annotation by its location in text
        locKey = (sentenceID, wordID)
        if (locKey not in annotationsByLoc[file]):
            annotationsByLoc[file][locKey] = []
        annotationsByLoc[file][locKey].append( [entityID, expression, annotation] )
        # Record annotation by its unique ID in text
        if (entityID not in annotationsByID[file]):
            annotationsByID[file][entityID] = []
        annotationsByID[file][entityID].append( [sentenceID, wordID, expression, annotation] )
    f.close()
    return (annotationsByLoc, annotationsByID)

def load_relation_annotation(inputFile): # VIIDATA
    annotationsByID  = dict()
    f = open(inputFile, mode='r', encoding="utf-8")
    for line in f:
        # Skip the comment line
        if ( re.match("^#.+$", line) ):
            continue
        items = line.split("\t")
        # old format: fileName	entityID_A	relation	entityID_B	comment	expression_A	expression_B
        # new format: fileName	entityID_A	relation	entityID_B	comment	
        if (len(items) != 5):
            print (len(items))
            raise Exception(" Unexpected number of items on line: '"+str(line)+"'")
        file     = items[0]
        entityA  = items[1]
        relation = items[2]
        entityB  = items[3]
        comment  = items[4].rstrip()
        if (file not in annotationsByID):
            annotationsByID[file] = dict()
        annotation = [entityA, relation, entityB, comment]
        if (entityA not in annotationsByID[file]):
            annotationsByID[file][entityA] = []
        annotationsByID[file][entityA].append( annotation )
        if (entityB not in annotationsByID[file]):
            annotationsByID[file][entityB] = []
        annotationsByID[file][entityB].append( annotation )
    f.close()
    return annotationsByID

def load_relation_to_dct_annotations(inputFile):
    annotationsByID  = dict()
    f = open(inputFile, mode='r', encoding="utf-8")
    for line in f:
        # Skip the comment line
        if ( re.match("^#.+$", line) ):
            continue
        items = line.split("\t")
        # old format: fileName	entityID_A	relation_to_DCT	comment	expression_A
        # new format: fileName	entityID_A	relation_to_DCT	comment
        if (len(items) != 4):
            raise Exception(" Unexpected number of items on line: '"+str(line)+"'")
        file          = items[0]
        entityA       = items[1]
        relationToDCT = items[2]
        comment       = items[3].rstrip()
        if (file not in annotationsByID):
            annotationsByID[file] = dict()
        annotation = [entityA, relationToDCT, "t0", comment]
        if (entityA not in annotationsByID[file]):
            annotationsByID[file][entityA] = []
        annotationsByID[file][entityA].append( annotation )
    f.close()
    return annotationsByID

# Artikklite DCT saamiseks
def load_articles_DCT(inputFile):
  articlesDCT = dict()
  f = open(inputFile, mode='r', encoding="utf-8")
  for line in f:
    # Skip the comment line
      if ( re.match("^#.+$", line) ):
          continue
      items = line.split("\t")
      article = items[0]
      info = items[1].split(" | ")
      for inf in info:
        if "ajalehenumber" in inf:
          match = re.search(r'(\d+\.\d+\.\d+)',inf)
          #print(match.group(1))
          articlesDCT[article] = match.group(1)
          break
  return articlesDCT

# Korpuse failide töötlemine

## Korpuse tekstidest EstNLTK 1.6 Text objektide loomine

In [8]:
baseAnnotations = load_base_segmentation(baseAnnotationFile)
(eventsByLoc, eventsByID) = load_entity_annotation(eventAnnotationFile)
articlesDCT = load_articles_DCT(articleMetadata_file)
sentence_events =  eventsByLoc.get('aja_ml_2002_47.tasak.a006.sol')
sentence_events =  eventsByID.get('aja_ml_2002_47.tasak.a006.sol')
(timexesByLoc, timexesByID) = load_entity_annotation(timexAnnotationFile)
sentence_timex = timexesByLoc.get('aja_ml_2002_47.tasak.a006.sol')

In [9]:
eventsByLoc

{'aja_ml_2002_47.tasak.a006.sol': {('1',
   '2'): [['e1', '"ehitab"', 'EVENT OCCURRENCE']],
  ('10', '1'): [['e21', '"vastu olid"', 'EVENT multiword="true"']],
  ('10', '10'): [['e22', '"asub"', 'EVENT STATE']],
  ('10', '2'): [['e21', '"vastu olid"', 'EVENT STATE multiword="true"']],
  ('11', '16'): [['e24', '"ehitatakse"', 'EVENT OCCURRENCE']],
  ('11', '9'): [['e23', '"imesta"', 'EVENT I_STATE polarity="NEG"']],
  ('12', '6'): [['e25', '"lastud"', 'EVENT OCCURRENCE']],
  ('13', '10'): [['e28', '"hakatakse"', 'EVENT ASPECTUAL']],
  ('13',
   '11'): [['e29', '"küsima"', 'EVENT I_ACTION comment="ka OCCURRENCE?"']],
  ('13', '18'): [['e30', '"lasite"', 'EVENT OCCURRENCE']],
  ('13',
   '24'): [['e31',
    '"pakkunud"',
    'EVENT OCCURRENCE polarity="NEG" comment="ka I_ACTION?"']],
  ('13', '27'): [['e32', '"ütles"', 'EVENT REPORTING']],
  ('13', '3'): [['e26', '"tulnuks"', 'EVENT MODAL']],
  ('13', '6'): [['e27', '"teha"', 'EVENT OCCURRENCE modality="tulema"']],
  ('14', '2'): [['e33',

### Kole testimise ja katsetamise kood

In [ ]:
# -------------------------------------------------------------------------------
# Kole testimise ja katsetamise kood siin. All pool olevas cell'is on ilusam asi.
# -------------------------------------------------------------------------------

#print(baseAnnotations.get('aja_ml_2002_47.tasak.a006.sol'))
sentence_base_annotations =  baseAnnotations.get('aja_ml_2002_47.tasak.a006.sol')
sentence_events =  eventsByLoc.get('aja_ml_2002_47.tasak.a006.sol')

salvestada = None
print(sentence_base_annotations)
print()
print(len(sentence_base_annotations))
sentence_id = 1
file_text = ""
for sentence in sentence_base_annotations:
  print(sentence)
  text = []
  lemmas = []
  #print(sentence[0][0])
  if sentence[0][0] == '0':
    continue
  for word_info in sentence:
    word = word_info[2]
    text.append(word)
    lemma = word_info[3].split(' ')[0].replace("\"", "")
    lemmas.append([lemma])
  text = " ".join(text)
  if file_text == "":
    file_text += text
  else:
    text = " " + text
    file_text += text
  #print("Failitekst: ", file_text)
  print(text)
  estnltk_text = estnltk.Text(text)
  estnltk_text.tag_layer(['words', 'sentences', 'morph_analysis'])
  #print(estnltk_text.layers)
  # EVENTS layer with event_ID and event annotation
  event_layer = estnltk.Layer(name='EVENTS',
                text_object=estnltk_text,
                attributes=['event_ID', 'event_annotation', 'event_class'])
  timex_layer = estnltk.Layer(name='TIMEX',
                text_object=estnltk_text,
                attributes=['expression', 'timex_annotation', 'type', 'value', 'timex_ID'])
  print("ennetsykklit")
  print(sentence_events.keys())
  for event in sentence_events.keys():
    if (int(event[0]) != sentence_id):
      continue
    #print("passed")
    #print("event ", event)
    print(sentence_events.get(event))
    value = sentence_events.get(event)[0]
    #print(event)
    #print(value)
    word_loc = int(event[1])
    word_start = estnltk_text['words'][word_loc].start
    word_end = estnltk_text['words'][word_loc].end
    #print(word_start, word_end)
    #print(word_end)
    #print(event)
    e_class = value[2].split()[1]
    event_layer.add_annotation( (word_start, word_end), event_ID=value[0], event_annotation=value[2] , event_class=e_class )
    # Event class event_annotationi alt
    #print("class", event_class)
    #print(estnltk_text.layers)
  estnltk_text.add_layer(event_layer)
  for timex in sentence_timex:
    if (int(timex[0]) != sentence_id):
      continue
    print("timexasi")
    value = sentence_timex.get(timex)[0]
    timex_ID = value[0]
    expression = value[1]
    timex_annotation = value[2]
    word_loc = int(timex[1])
    word_start = estnltk_text['words'][word_loc].start
    word_end = estnltk_text['words'][word_loc].end
    timex_type = timex_annotation.split()[1]
    #timex_value = timex_annotation.split()[]
    timex_layer.add_annotation((word_start, word_end), timex_ID=timex_ID, expression=expression, timex_annotation=timex_annotation, type=None , value=None)
    #value = sentence_events.get(event)[0]
  estnltk_text.add_layer(timex_layer)
  #print(estnltk_text.layers)
  pos = estnltk_text['words'].lemma
  pos = lemmas
  #print(estnltk_text['morph_analysis']['lemma'])
  #print(lemmas)
  #estnltk_text['morph_analysis']['lemma'] = lemmas
  #print("Lemmad")
  #print(estnltk_text.lemma)
  #print(pos)
  print(estnltk_text)
  print(estnltk_text['words'].text)
  print(estnltk_text.layers)
  #print(estnltk_text['words'].lemma)
  #print(estnltk_text.layers)

  if 'EVENTS' in estnltk_text.layers:
    print("EVENTS olemas")
    print(estnltk_text['EVENTS'])
    print(estnltk_text['EVENTS'].event_ID)
    print(estnltk_text['EVENTS'].event_annotation)
  if 'TIMEX' in estnltk_text.layers:
    print("TIMEX olemas")
    print(estnltk_text['TIMEX'])
    print(estnltk_text['TIMEX'].timex_ID)
    print(estnltk_text['TIMEX'].expression)
    print(estnltk_text['TIMEX'].timex_annotation)
  if len(estnltk_text['TIMEX'].timex_ID) > 0 and len(estnltk_text['EVENTS'].event_ID) > 0 and 'e11' in estnltk_text['EVENTS'].event_ID:
    print("-------------", estnltk_text['EVENTS'].event_ID)
    salvestada = estnltk_text
  print("--")
  sentence_id += 1
print()
print("alldone")
print(salvestada)
salvestada



[[['0', '0', '20.11.2002', '"20.11.2002" L0 N card <?> digit @ADVL', '1', '0']], [['1', '0', 'Tallinna', '"Tallinn" L0 S prop sg gen cap @NN>', '1', '2'], ['1', '1', 'Sadam', '"sadam" L0 S com sg nom cap @SUBJ', '2', '3'], ['1', '2', 'ehitab', '"ehita" Lb V main indic pres ps3 sg ps af <FinV> <NGP-P> <All> @FMV', '3', '0'], ['1', '3', 'Saaremaa', '"Saare_maa" L0 S prop sg gen cap @NN>', '4', '5'], ['1', '4', 'süvasadama', '"süva_sadam" L0 S com sg gen @OBJ', '5', '3'], ['1', '5', 'Küdema', '"Küdema" L0 S prop sg gen @NN>', '6', '7'], ['1', '6', 'lahe', '"laht" L0 S com sg gen @NN>', '7', '8'], ['1', '7', 'läänerannikule', '"lääne_rannik" Lle S com sg all @ADVL', '8', '3'], ['1', '8', 'endisesse', '"endine" Lsse A pos sg ill @AN>', '9', '11'], ['1', '9', 'Tamme', '"tamm" L0 S com sg gen cap @NN>', '10', '11'], ['1', '10', 'sadamakohta', '"sadama_koht" L0 S com sg adit @ADVL', '11', '3'], ['1', '11', 'Mustjala', '"Must_jala" L0 S prop sg gen cap @NN>', '12', '13'], ['1', '12', 'valda', '

Text(text=' Tammel oli kalasadam veel 1960. aastate lõpul , enne uue kalasadama rajamist Tagalahte Veerele .')

In [ ]:
salvestada

Text(text=' Tammel oli kalasadam veel 1960. aastate lõpul , enne uue kalasadama rajamist Tagalahte Veerele .')

In [ ]:
salvestada.EVENTS

Layer(name='EVENTS', attributes=('event_ID', 'event_annotation', 'event_class'), spans=SL[Span('oli', [{'event_ID': 'e11', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('rajamist', [{'event_ID': 'e12', 'event_annotation': 'EVENT OCCURRENCE comment="märgendamine vaieldav sügavuse tõttu"', 'event_class': 'OCCURRENCE'}])])

In [ ]:
salvestada.TIMEX

layer name,attributes,parent,enveloping,ambiguous,span count
TIMEX,"expression, timex_annotation, type, value, timex_ID",None,None,False,3
text,expression,timex_annotation,type,value,timex_ID
1960.,"""1960. aastate lõpul""","TIMEX DATE 196 mod=""END"" multiword=""true""",None,None,t2
aastate,"""1960. aastate lõpul""","TIMEX multiword=""true""",None,None,t2
lõpul,"""1960. aastate lõpul""","TIMEX multiword=""true""",None,None,t2


In [ ]:
sentence_base_annotations

[[['0',
   '0',
   '20.11.2002',
   '"20.11.2002" L0 N card <?> digit @ADVL',
   '1',
   '0']],
 [['1', '0', 'Tallinna', '"Tallinn" L0 S prop sg gen cap @NN>', '1', '2'],
  ['1', '1', 'Sadam', '"sadam" L0 S com sg nom cap @SUBJ', '2', '3'],
  ['1',
   '2',
   'ehitab',
   '"ehita" Lb V main indic pres ps3 sg ps af <FinV> <NGP-P> <All> @FMV',
   '3',
   '0'],
  ['1', '3', 'Saaremaa', '"Saare_maa" L0 S prop sg gen cap @NN>', '4', '5'],
  ['1', '4', 'süvasadama', '"süva_sadam" L0 S com sg gen @OBJ', '5', '3'],
  ['1', '5', 'Küdema', '"Küdema" L0 S prop sg gen @NN>', '6', '7'],
  ['1', '6', 'lahe', '"laht" L0 S com sg gen @NN>', '7', '8'],
  ['1',
   '7',
   'läänerannikule',
   '"lääne_rannik" Lle S com sg all @ADVL',
   '8',
   '3'],
  ['1', '8', 'endisesse', '"endine" Lsse A pos sg ill @AN>', '9', '11'],
  ['1', '9', 'Tamme', '"tamm" L0 S com sg gen cap @NN>', '10', '11'],
  ['1',
   '10',
   'sadamakohta',
   '"sadama_koht" L0 S com sg adit @ADVL',
   '11',
   '3'],
  ['1', '11', 'Must

### Korralikum kood EstNLTK Text objektide loomiseks
Lisab terve artikli teksti ühte Text objekti. Lisan kaks *layer*it sündmuste (*gold_events*) ja ajamäärangute (*gold_timexes*) jaoks. Samuti Text objekti metaandmetesse lisan dokumendi loomise aja (*DCT*).  
Loon ka *gold_word_events* kihi (IOB formaadis märgendused).  
*(sentence_nr, word_nr) -> (start, end) -> word_id in 'words'* mappingu jaoks loon ka sõnastiku 'mapping'.


In [ ]:
# -------------------------------------------------------------------------------
# Korralikum kood
# -------------------------------------------------------------------------------
def create_Text_object(filename):
  event_classes = ['REPORTING', 'PERCEPTION', 'ASPECTUAL', 'I_ACTION', 'I_STATE', 'STATE', 'MODAL', 'OCCURRENCE', 'EVENT_CONTAINER', 'CAUSE']
  timex_types = ['DATE', 'TIME', 'DURATION', 'SET']

  #print(baseAnnotations.get('aja_ml_2002_47.tasak.a006.sol')) # aja_sloleht_2007_12_17.tasak.a044.sol, aja_sloleht_2007_12_17.tasak.a051.sol
  #filename = 'aja_ml_2002_47.tasak.a006.sol'
  sentence_base_annotations =  baseAnnotations.get(filename)
  sentence_events =  eventsByLoc.get(filename)
  sentence_timex = timexesByLoc.get(filename)
  #print(type(sentence_events), type(sentence_timex))

  sentence_id = 0 # mapping
  word_id_in_sentence = 0 # mapping
  letter_index = 0 # mapping
  mapping = {} # mapping
  # Going through the article and making one estnltk Text object with all sentences
  file_text = ""

  last_event_class = None
  last_timex_type = None
  last_timex_value = None
  for sentence in sentence_base_annotations:
    text = []
    for word_info in sentence:
      word = word_info[2]
      letter_index += len(word) # mapping
      text.append(word)
      #print(sentence_id, word_id_in_sentence)
      #print(word)
      mapping[(sentence_id, word_id_in_sentence)] = (letter_index - len(word), letter_index) # mapping
      word_id_in_sentence += 1 # mapping
      letter_index += 1 # mapping (kuna sõnade vahel on tühikud)

    text = " ".join(text)
    if file_text == "":
      file_text += text
    else:
      text = " " + text
      file_text += text
    sentence_id += 1 # mapping
    word_id_in_sentence = 0 # mapping
  article_text = estnltk.Text(file_text)
  article_text.tag_layer(['words', 'sentences', 'morph_analysis'])
  # Making a layer for events and timexes
  event_layer = estnltk.Layer(name='gold_events',
                  text_object=article_text,
                  attributes=['sentence_ID', 'word_ID_in_sentence','event_ID', 'expression', 'event_annotation', 'event_class'])
  timex_layer = estnltk.Layer(name='gold_timexes',
                  text_object=article_text,
                  attributes=['sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'],
                  ambiguous=True)
  gold_word_events = estnltk.Layer(name="gold_word_events", text_object=article_text, attributes=['nertag', 'sentence_ID'], enveloping='words') # gold_word_events

  # Start going over sentences in the article
  sentence_nr = 0
  last_word_was_multiword = False
  last_event_tag = None

  word_is_multiword = False
  multiword_event_tags = set()
  ##for sent in article_text.sentences:
  for sentence in sentence_base_annotations:##
    word_nr = 0
    ##for word in sent:
    for word_info in sentence:##
      word = word_info[2]##
      #print(word)
      startend = mapping.get((sentence_nr, word_nr))##
      #print(startend)
      event = sentence_events.get((str(sentence_nr), str(word_nr)))
      if sentence_timex:
        timex = sentence_timex.get((str(sentence_nr), str(word_nr)))
      else:
        timex = None
      #print(timex)
      # Sõna on EVENT
      if event:
        #print(event[0][0])
        e_class = event[0][2].split()[1]
        is_multiword = 'multiword="true"' in event[0][2].split()
        multiword_main_word = event[0][2].split()[1] != 'multiword="true"'
        event_tag = event[0][0]
        #print(event[0][2])
        # gold_word_events taggide otsustamine

        if event_tag != last_event_tag and event_tag not in multiword_event_tags:
          gold_word_events.add_annotation((startend[0], startend[1]), nertag="B-EVENT", sentence_ID=sentence_nr) # gold_word_events
        else:
          gold_word_events.add_annotation((startend[0], startend[1]), nertag="I-EVENT", sentence_ID=sentence_nr) # gold_word_events
        
        if is_multiword:
          multiword_event_tags.add(event_tag)

        last_event_tag = event_tag
        #if is_multiword and last_word_was_multiword:
        #  gold_word_events.add_annotation((startend[0], startend[1]), nertag="I-EVENT", sentence_ID=sentence_nr) # gold_word_events
        #else:
        #  gold_word_events.add_annotation((startend[0], startend[1]), nertag="B-EVENT", sentence_ID=sentence_nr) # gold_word_events
        #  if is_multiword:
        #    last_word_was_multiword = True
        #  else:
        #    last_word_was_multiword = False
        # Kui sellist event_class'i pole, siis tõenäoliselt on tegemist multiword osaga
        if e_class not in event_classes:
          e_class = event[0][2].split()[0] # multiword puhul on class teisel kohal
        event_layer.add_annotation((startend[0], startend[1]), sentence_ID=sentence_nr, word_ID_in_sentence=word_nr, event_ID=event[0][0], expression=event[0][1], event_annotation=event[0][2], event_class=e_class)
      # Sõna on TIMEX
      elif timex:
        last_word_was_multiword = False
        #print("he")
        #print(timex)
        gold_word_events.add_annotation((startend[0], startend[1]), nertag="O", sentence_ID=sentence_nr) # gold_word_events
        ##t_type = timex[0][2].split()[1]
        # Kui sellist timex_type'i pole, siis tõenäoliselt on tegemist multiword osaga
        # Fraasi esimese sõna atribuutide väärtused kehtivad üle fraasi ka teistel sõnadel
        ##if t_type in timex_types:
        ##  last_timex_type = t_type
        ##  t_value = timex[0][2].split()[2]
        ##  last_timex_value = t_value
        ##else: # on multiword osa
        ##  t_type = last_timex_type
        ##  t_value = last_timex_value
        ##timex_layer.add_annotation((startend[0], startend[1]), sentence_ID=sentence_nr, word_ID_in_sentence=word_nr, timex_ID=timex[0][0], expression=timex[0][1], timex_annotation=timex[0][2], type=t_type, value=t_value)
        

        # timex võib sisaldada ka mitut tähistust (tokenSubstringi puhul)
        # et span start ja end'iga ei tekiks konflikte, tuleb leida mis maalt hakkab järgmine tokenSubstring
        # Näide
        #--- [['t2', '"29. novembril kl 11-"', 'TIMEX multiword="true" tokenSubstring="true"']]
        #--- sõna: 11-17
        #--- start ja end: (28, 33)
        #--- [['t2', '"29. novembril kl 11-"', 'TIMEX multiword="true" tokenSubstring="true"'], ['t3', '"17"', 'TIMEX TIME 2002-11-29T17 tokenSubstring="true"']]
        if(len(timex) > 2):
          print("SUUREM KUI", filename)
          print("aa" - "a")
        token_start = startend[0]
        token_end = startend[1]
        #print(token_start, token_end)
        for i in range(len(timex)):
          t_type = timex[i][2].split()[1]
          # Kui sellist timex_type'i pole, siis tõenäoliselt on tegemist multiword osaga
          # Fraasi esimese sõna atribuutide väärtused kehtivad üle fraasi ka teistel sõnadel
          if t_type in timex_types:
            last_timex_type = t_type
            t_value = timex[i][2].split()[2]
            last_timex_value = t_value
          else: # on multiword osa
            t_type = last_timex_type
            t_value = last_timex_value
          # Tuleb starti ja endi muuta
          if i > 0:
            # wordi sees on expression
            expression = timex[i][1].strip('""')
            #liita = word.find(expression)
            #print("sõna", word)
            #print("otsin algust:", expression)
            #print(word.find(expression))
            if word.find(expression) == -1:
              if word == expression:
                liita = token_start
              else:
                #print("sin", len(expression), expression)
                liita = len(timex[i-1][1].strip('""'))
            #print("liidan", liita)
            #token_start += liita
            #token_end = token_start + len(expression)
            #print("sain", token_start, token_end)
          #print("exp", timex[i][1])
          #print("start", token_start, "end", token_end, timex[i][1])
          timex_layer.add_annotation((token_start, token_end), sentence_ID=sentence_nr, word_ID_in_sentence=word_nr, timex_ID=timex[i][0], expression=timex[i][1], timex_annotation=timex[i][2], type=t_type, value=t_value)


      else:
        last_word_was_multiword = False
        gold_word_events.add_annotation((startend[0], startend[1]), nertag="O", sentence_ID=sentence_nr) # gold_word_events
      word_nr += 1
    sentence_nr += 1


  time = articlesDCT.get(filename).split(".")
  if len(time[2]) == 4:
    time = time[2] + "-" + time[1] + "-" + time[0]
  else:
    time = time[0] + "-" + time[1] + "-" + time[2]

  # Adding layers to article text
  article_text.meta['dct'] = articlesDCT.get(filename)
  article_text.meta['creation_time'] = time
  article_text.meta['filename'] = filename
  article_text.add_layer(event_layer)
  article_text.add_layer(timex_layer)
  article_text.add_layer(gold_word_events)
  
  return article_text, mapping

In [ ]:
article_text, mapping = create_Text_object('aja_ml_2002_47.tasak.a023.sol')
#aja_pm_2000_10_04.tasak.a015.sol
#aja_ml_2002_47.tasak.a016.sol
#aja_ml_2002_47.tasak.a038.sol
#
article_text

Text(text='20.11.2002 Juba aastaid on piimavarumisel kestnud anarhia . Kui vahepeal tundubki , et suuremad hädad hakkavad mööda saama , tuleb uus löök . See , et Rapla Dairy on lehmapidajatele kuude kaupa võlgu , pälvis laiemat tähelepanu alles siis , kui riik tahtis firmat juhtivale Afganistani eruohvitserile Jaqub Haidaryle eriliste teenete eest kodakondsuse kinkida . Mure suuruse võtab oma ilmekas lihtsuses kokku põllumajandusministeeriumile saadetud abipalve . Selle kirjutasid kaheksa Noarootsi lehmapidajat , kel piimaraha seitse kuud saamata . Kirjas seisab : " Võimaluse piires palume tasuta abi juristi või advokaadi näol , kes tunneks seadusi ja oskaks meid , vaeseid piimatootjaid , kaitsta . " Mida kuude kaupa saamata piimaraha tähendab , pole vaja seletada maainimesele , kellele see on üks põhilisi elatusallikaid . Mis kõige hullem - need inimesed on oma murega sisuliselt üksi , sest need , kes peaksid abi pakkuma , leiavad alati tuhat põhjendust , miks aidata pole võimalik . Põllumajandusministeerium püüab lahendusi otsida piimasektorile kiiranalüüsi tehes . Tekib küsimus , kas ülevaadet piimanduses toimuvast siis ministeeriumil varem polnudki . Kas senistest probleemidest pole õpitud ? Ja kas see hinnang tuleb ikka objektiivne või püütakse seisu euroläbirääkimiste taustal ilustada . Teised piimatööstused vaevalt Dairyt ja tema eelkäijat Lactot , kes kunagi sundis neid kokkuostuhinda kergitama , taga nutavad . Üks konkurent vähem . Pank muretseb jälle , kuidas oma laenu tagasi saada . Peaminister leiab , et rahaline abi raskustes firmadele on raiskamine . Aga maal oodatakse ja oodatakse piimaraha . Kuigi praegu töötatakse välja skeemi Dairy võlgade katmiseks , ei saa Dairyt vaadata ainult üksikjuhtumina . Kogu süsteem lonkab , ainult ei jõuta kokkuleppele , kumb jalg on haige või on tegu hoopis psüühikahäirega . Teistes , arenenumates riikides , kuuluvad piimatööstused enamasti ühistutele , mis on tootjate endi omad . Meil kunagi õitsval järjel olnud ühistegevust pole aga riik piisavalt toetada tahtnud . Et suuremad ärid õitseksid ja raha liiga laiali ei pudeneks . Ühe või paari lehma pidamine tundub selle kõrval ju nii tühine . Ent asi pole mitte lehmades , vaid inimestes . Kui asja kas või puhtmajanduslikult võtta , siis kumb on otstarbekam : kas maksta vaesematele maainimestele näljapajukit sotsiaaltoetuste näol või anda neile võimalus ise toime tulla . Läbi töö , mida nad oskavad ja austavad . Ka ilmset ebaõiglust võetakse tänases Eestis järjest enam loomuliku nähtusena . Nii ei jäägi talumehel ühel heal päeval muud üle , kui viimane lehm käekõrvale võtta ja advokaadi ukse taha viia . Piimaraha saamata , tapamajas järjekord , maksan natuuras .')

In [ ]:
article_text.gold_events

Layer(name='gold_events', attributes=('sentence_ID', 'word_ID_in_sentence', 'event_ID', 'expression', 'event_annotation', 'event_class'), spans=SL[Span('piimavarumisel', [{'sentence_ID': 1, 'word_ID_in_sentence': 3, 'event_ID': 'e1', 'expression': '"piimavarumisel"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kestnud', [{'sentence_ID': 1, 'word_ID_in_sentence': 4, 'event_ID': 'e2', 'expression': '"kestnud"', 'event_annotation': 'EVENT ASPECTUAL comment="fraas: kestnud anarhia"', 'event_class': 'ASPECTUAL'}]),
Span('anarhia', [{'sentence_ID': 1, 'word_ID_in_sentence': 5, 'event_ID': 'e3', 'expression': '"anarhia"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tundubki', [{'sentence_ID': 2, 'word_ID_in_sentence': 2, 'event_ID': 'e4', 'expression': '"tundubki"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('hakkavad', [{'sentence_ID': 2, 'word_ID_in_sentence': 7, 'event_ID': 'e5', 'expression': '"hakkavad"', 'event_annotation': 'EVENT ASPECTUAL', 'event_class': 'ASPECTUAL'}]),
Span('saama', [{'sentence_ID': 2, 'word_ID_in_sentence': 9, 'event_ID': 'e6', 'expression': '"saama"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tuleb', [{'sentence_ID': 2, 'word_ID_in_sentence': 11, 'event_ID': 'e7', 'expression': '"tuleb"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('on', [{'sentence_ID': 3, 'word_ID_in_sentence': 5, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT STATE multiword="true"', 'event_class': 'STATE'}]),
Span('võlgu', [{'sentence_ID': 3, 'word_ID_in_sentence': 9, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT multiword="true"', 'event_class': 'EVENT'}]),
Span('pälvis', [{'sentence_ID': 3, 'word_ID_in_sentence': 11, 'event_ID': 'e9', 'expression': '"pälvis"', 'event_annotation': 'EVENT OCCURRENCE comment="ka I_STATE: see, et on võlgu, pälvis tähelepanu"', 'event_class': 'OCCURRENCE'}]),
Span('tahtis', [{'sentence_ID': 3, 'word_ID_in_sentence': 19, 'event_ID': 'e10', 'expression': '"tahtis"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('teenete', [{'sentence_ID': 3, 'word_ID_in_sentence': 27, 'event_ID': 'e11', 'expression': '"teenete"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kinkida', [{'sentence_ID': 3, 'word_ID_in_sentence': 30, 'event_ID': 'e12', 'expression': '"kinkida"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('võtab', [{'sentence_ID': 4, 'word_ID_in_sentence': 2, 'event_ID': 'e13', 'expression': '"võtab"', 'event_annotation': 'EVENT STATE comment="fraas: võtab kokku"', 'event_class': 'STATE'}]),
Span('saadetud', [{'sentence_ID': 4, 'word_ID_in_sentence': 8, 'event_ID': 'e14', 'expression': '"saadetud"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kirjutasid', [{'sentence_ID': 5, 'word_ID_in_sentence': 1, 'event_ID': 'e15', 'expression': '"kirjutasid"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('saamata', [{'sentence_ID': 5, 'word_ID_in_sentence': 10, 'event_ID': 'e16', 'expression': '"saamata"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('seisab', [{'sentence_ID': 6, 'word_ID_in_sentence': 1, 'event_ID': 'e17', 'expression': '"seisab"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('palume', [{'sentence_ID': 6, 'word_ID_in_sentence': 6, 'event_ID': 'e18', 'expression': '"palume"', 'event_annotation': 'EVENT I_ACTION', 'event_class': 'I_ACTION'}]),
Span('abi', [{'sentence_ID': 6, 'word_ID_in_sentence': 8, 'event_ID': 'e19', 'expression': '"abi"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tunneks', [{'sentence_ID': 6, 'word_ID_in_sentence': 15, 'event_ID': 'e20', 'expression': '"tunneks"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('oskaks', 

## Paranda gold_events class väärtused
"multiword=true" puhul on event_classiks vahest "EVENT".  
Käin gold_events läbi. Kui class == "EVENT", vaatan ümbritsevaid sündmusi sama IDga ja muudan classi sobivaks.

In [ ]:
def paranda_g_event_classid(tekst_objekt):
  for i in range(len(tekst_objekt.gold_events)):
    if tekst_objekt.gold_events[i].event_class == "EVENT":
      event_ID = tekst_objekt.gold_events[i].event_ID
      # Vaata eelnevaid
      for j in range(i - 1, -1, -1):
        # On sama sündmus ja class on õige
        if event_ID == tekst_objekt.gold_events[j].event_ID and tekst_objekt.gold_events[j].event_class != "EVENT":
          tekst_objekt.gold_events[i].event_class = tekst_objekt.gold_events[j].event_class
          break
      # Vaatan tulevaid
      for j in range(i + 1, len(tekst_objekt.gold_events)):
        if event_ID == tekst_objekt.gold_events[j].event_ID and tekst_objekt.gold_events[j].event_class != "EVENT":
          tekst_objekt.gold_events[i].event_class = tekst_objekt.gold_events[j].event_class
          break

paranda_g_event_classid(article_text)
article_text.gold_events

Layer(name='gold_events', attributes=('sentence_ID', 'word_ID_in_sentence', 'event_ID', 'expression', 'event_annotation', 'event_class'), spans=SL[Span('piimavarumisel', [{'sentence_ID': 1, 'word_ID_in_sentence': 3, 'event_ID': 'e1', 'expression': '"piimavarumisel"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kestnud', [{'sentence_ID': 1, 'word_ID_in_sentence': 4, 'event_ID': 'e2', 'expression': '"kestnud"', 'event_annotation': 'EVENT ASPECTUAL comment="fraas: kestnud anarhia"', 'event_class': 'ASPECTUAL'}]),
Span('anarhia', [{'sentence_ID': 1, 'word_ID_in_sentence': 5, 'event_ID': 'e3', 'expression': '"anarhia"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tundubki', [{'sentence_ID': 2, 'word_ID_in_sentence': 2, 'event_ID': 'e4', 'expression': '"tundubki"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('hakkavad', [{'sentence_ID': 2, 'word_ID_in_sentence': 7, 'event_ID': 'e5', 'expression': '"hakkavad"', 'event_annotation': 'EVENT ASPECTUAL', 'event_class': 'ASPECTUAL'}]),
Span('saama', [{'sentence_ID': 2, 'word_ID_in_sentence': 9, 'event_ID': 'e6', 'expression': '"saama"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tuleb', [{'sentence_ID': 2, 'word_ID_in_sentence': 11, 'event_ID': 'e7', 'expression': '"tuleb"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('on', [{'sentence_ID': 3, 'word_ID_in_sentence': 5, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT STATE multiword="true"', 'event_class': 'STATE'}]),
Span('võlgu', [{'sentence_ID': 3, 'word_ID_in_sentence': 9, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT multiword="true"', 'event_class': 'STATE'}]),
Span('pälvis', [{'sentence_ID': 3, 'word_ID_in_sentence': 11, 'event_ID': 'e9', 'expression': '"pälvis"', 'event_annotation': 'EVENT OCCURRENCE comment="ka I_STATE: see, et on võlgu, pälvis tähelepanu"', 'event_class': 'OCCURRENCE'}]),
Span('tahtis', [{'sentence_ID': 3, 'word_ID_in_sentence': 19, 'event_ID': 'e10', 'expression': '"tahtis"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('teenete', [{'sentence_ID': 3, 'word_ID_in_sentence': 27, 'event_ID': 'e11', 'expression': '"teenete"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kinkida', [{'sentence_ID': 3, 'word_ID_in_sentence': 30, 'event_ID': 'e12', 'expression': '"kinkida"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('võtab', [{'sentence_ID': 4, 'word_ID_in_sentence': 2, 'event_ID': 'e13', 'expression': '"võtab"', 'event_annotation': 'EVENT STATE comment="fraas: võtab kokku"', 'event_class': 'STATE'}]),
Span('saadetud', [{'sentence_ID': 4, 'word_ID_in_sentence': 8, 'event_ID': 'e14', 'expression': '"saadetud"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kirjutasid', [{'sentence_ID': 5, 'word_ID_in_sentence': 1, 'event_ID': 'e15', 'expression': '"kirjutasid"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('saamata', [{'sentence_ID': 5, 'word_ID_in_sentence': 10, 'event_ID': 'e16', 'expression': '"saamata"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('seisab', [{'sentence_ID': 6, 'word_ID_in_sentence': 1, 'event_ID': 'e17', 'expression': '"seisab"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('palume', [{'sentence_ID': 6, 'word_ID_in_sentence': 6, 'event_ID': 'e18', 'expression': '"palume"', 'event_annotation': 'EVENT I_ACTION', 'event_class': 'I_ACTION'}]),
Span('abi', [{'sentence_ID': 6, 'word_ID_in_sentence': 8, 'event_ID': 'e19', 'expression': '"abi"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tunneks', [{'sentence_ID': 6, 'word_ID_in_sentence': 15, 'event_ID': 'e20', 'expression': '"tunneks"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('oskaks', 

## gold_word_events_w_classes

In [ ]:
def add_gold_word_events_w_classes(tekst_objekt):
  indeks = 0
  gold_word_events_w_classes = estnltk.Layer(name="gold_word_events_w_classes", text_object=tekst_objekt, attributes=['nertag', 'sentence_ID'], enveloping='words')
  for gold_word_event in tekst_objekt.gold_word_events:
    e_class = tekst_objekt.gold_events
    if gold_word_event.nertag != "O":
      e_class = tekst_objekt.gold_events[indeks].event_class
      indeks += 1
      gold_word_events_w_classes.add_annotation(gold_word_event.base_span, nertag=gold_word_event.nertag + "_" + e_class, sentence_ID=gold_word_event.sentence_ID)
    else:
      gold_word_events_w_classes.add_annotation(gold_word_event.base_span, nertag=gold_word_event.nertag, sentence_ID=gold_word_event.sentence_ID)
  tekst_objekt.add_layer(gold_word_events_w_classes)



add_gold_word_events_w_classes(article_text)
article_text.gold_word_events_w_classes

Layer(name='gold_word_events_w_classes', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
Envelopi

## gold_word_events_only_main

In [ ]:
def add_gold_word_events_only_main(tekst_objekt):
  event_classes = ['REPORTING', 'PERCEPTION', 'ASPECTUAL', 'I_ACTION', 'I_STATE', 'STATE', 'MODAL', 'OCCURRENCE', 'EVENT_CONTAINER', 'CAUSE']

  gold_word_events_only_main = estnltk.Layer(name="gold_word_events_only_main", text_object=tekst_objekt, attributes=['nertag', 'sentence_ID'], enveloping='words')
  indeks = 0
  for gold_word_event in tekst_objekt.gold_word_events:
    if gold_word_event.nertag != "O":
      e_class = tekst_objekt.gold_events[indeks].event_class
      event_annotation = tekst_objekt.gold_events[indeks].event_annotation
      indeks += 1
      #print(event_annotation, event_annotation.split(' ')[1] != 'multiword="true"')
      if event_annotation.split(' ')[1] != 'multiword="true"':
        gold_word_events_only_main.add_annotation(gold_word_event.base_span, nertag="B-EVENT" + "_" + e_class, sentence_ID=gold_word_event.sentence_ID)
      else:
        gold_word_events_only_main.add_annotation(gold_word_event.base_span, nertag="O", sentence_ID=gold_word_event.sentence_ID)
    else:
      gold_word_events_only_main.add_annotation(gold_word_event.base_span, nertag=gold_word_event.nertag, sentence_ID=gold_word_event.sentence_ID)


  tekst_objekt.add_layer(gold_word_events_only_main)

add_gold_word_events_only_main(article_text)
article_text.gold_word_events_only_main

Layer(name='gold_word_events_only_main', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilme

## gold_word_events synced with Text.words

In [ ]:
def add_gold_word_events_synced(tekst_objekt):
  gold_word_events_synced = estnltk.Layer(name="gold_word_events_synced", text_object=tekst_objekt, attributes=['nertag', 'sentence_ID'], enveloping='words')

  algus = 0
  osadega = False
  for i, word in enumerate(tekst_objekt.words):
    if word.text in tekst_objekt.gold_word_events[i-algus].text:
      gold_word_events_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events[i-algus].sentence_ID)
      if word.text != tekst_objekt.gold_word_events[i-algus].text:
        osadega = True
        algus += 1
      try:
        if osadega and tekst_objekt.words[i+1].text in tekst_objekt.gold_word_events[i+2-algus].text:
          osadega = False
          algus -= 1
      except:
        print("Expect:", len(tekst_objekt.words), i+1)
        print("Expect:", len(tekst_objekt.gold_word_events),i+2-algus)
    elif tekst_objekt.gold_word_events[i-algus].text in word.text:
      gold_word_events_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events[i-algus].sentence_ID)
      for j in range(i-algus, len(tekst_objekt.gold_word_events)):
        if tekst_objekt.gold_word_events[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break
    else:
      vahe = abs(word.start - tekst_objekt.gold_word_events[i-algus].end)
      gold_word_events_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events[i-algus].sentence_ID)
      if word.text[:vahe] == tekst_objekt.gold_word_events[i-algus].text[-vahe:]:
        algus -= 1
      for j in range(i-algus, len(tekst_objekt.gold_word_events)):
        if tekst_objekt.gold_word_events[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break

  tekst_objekt.add_layer(gold_word_events_synced)

add_gold_word_events_synced(article_text)
article_text.gold_word_events_synced

Layer(name='gold_word_events_synced', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilmekas', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('lihtsuses', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSp

## gold_word_events_w_classes synced with Text.words

In [ ]:
def add_gold_word_events_w_classes_synced(tekst_objekt):
  gold_word_events_w_classes_synced = estnltk.Layer(name="gold_word_events_w_classes_synced", text_object=tekst_objekt, attributes=['nertag', 'sentence_ID'], enveloping='words')

  algus = 0
  osadega = False
  for i, word in enumerate(tekst_objekt.words):
    if word.text in tekst_objekt.gold_word_events_w_classes[i-algus].text:
      gold_word_events_w_classes_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_w_classes[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_w_classes[i-algus].sentence_ID)
      if word.text != tekst_objekt.gold_word_events_w_classes[i-algus].text:
        osadega = True
        algus += 1
      try:
        if osadega and tekst_objekt.words[i+1].text in tekst_objekt.gold_word_events_w_classes[i+2-algus].text:
          osadega = False
          algus -= 1
      except:
        print("Expect:", len(tekst_objekt.words), i+1)
        print("Expect:", len(tekst_objekt.gold_word_events_w_classes),i+2-algus)
    elif tekst_objekt.gold_word_events_w_classes[i-algus].text in word.text:
      gold_word_events_w_classes_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_w_classes[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_w_classes[i-algus].sentence_ID)
      for j in range(i-algus, len(tekst_objekt.gold_word_events_w_classes)):
        if tekst_objekt.gold_word_events_w_classes[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break
    else:
      vahe = abs(word.start - tekst_objekt.gold_word_events_w_classes[i-algus].end)
      gold_word_events_w_classes_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_w_classes[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_w_classes[i-algus].sentence_ID)
      if word.text[:vahe] == tekst_objekt.gold_word_events_w_classes[i-algus].text[-vahe:]:
        algus -= 1
      for j in range(i-algus, len(tekst_objekt.gold_word_events_w_classes)):
        if tekst_objekt.gold_word_events_w_classes[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break

  tekst_objekt.add_layer(gold_word_events_w_classes_synced)

add_gold_word_events_w_classes_synced(article_text)
article_text.gold_word_events_w_classes_synced

Layer(name='gold_word_events_w_classes_synced', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
E

## gold_word_events_only_main synced with Text.words

In [ ]:
def add_gold_word_events_only_main_synced(tekst_objekt):
  gold_word_events_only_main_synced = estnltk.Layer(name="gold_word_events_only_main_synced", text_object=tekst_objekt, attributes=['nertag', 'sentence_ID'], enveloping='words')

  algus = 0
  osadega = False
  for i, word in enumerate(tekst_objekt.words):
    if word.text in tekst_objekt.gold_word_events_only_main[i-algus].text:
      gold_word_events_only_main_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_only_main[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_only_main[i-algus].sentence_ID)
      if word.text != tekst_objekt.gold_word_events_only_main[i-algus].text:
        osadega = True
        algus += 1
      try:
        if osadega and tekst_objekt.words[i+1].text in tekst_objekt.gold_word_events_only_main[i+2-algus].text:
          osadega = False
          algus -= 1
      except:
        print("Expect:", len(tekst_objekt.words), i+1)
        print("Expect:", len(tekst_objekt.gold_word_events_only_main),i+2-algus)
    elif tekst_objekt.gold_word_events_only_main[i-algus].text in word.text:
      gold_word_events_only_main_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_only_main[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_only_main[i-algus].sentence_ID)
      for j in range(i-algus, len(tekst_objekt.gold_word_events_only_main)):
        if tekst_objekt.gold_word_events_only_main[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break
    else:
      vahe = abs(word.start - tekst_objekt.gold_word_events_only_main[i-algus].end)
      gold_word_events_only_main_synced.add_annotation(word.base_span, nertag=tekst_objekt.gold_word_events_only_main[i-algus].nertag, sentence_ID=tekst_objekt.gold_word_events_only_main[i-algus].sentence_ID)
      if word.text[:vahe] == tekst_objekt.gold_word_events_only_main[i-algus].text[-vahe:]:
        algus -= 1
      for j in range(i-algus, len(tekst_objekt.gold_word_events_only_main)):
        if tekst_objekt.gold_word_events_only_main[j].text in word.text:
          algus -= 1
        else:
          algus += 1
          break

  tekst_objekt.add_layer(gold_word_events_only_main_synced)

add_gold_word_events_only_main_synced(article_text)
article_text.gold_word_events_only_main_synced

Layer(name='gold_word_events_only_main_synced', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpa

In [ ]:
article_text

Text(text='20.11.2002 Juba aastaid on piimavarumisel kestnud anarhia . Kui vahepeal tundubki , et suuremad hädad hakkavad mööda saama , tuleb uus löök . See , et Rapla Dairy on lehmapidajatele kuude kaupa võlgu , pälvis laiemat tähelepanu alles siis , kui riik tahtis firmat juhtivale Afganistani eruohvitserile Jaqub Haidaryle eriliste teenete eest kodakondsuse kinkida . Mure suuruse võtab oma ilmekas lihtsuses kokku põllumajandusministeeriumile saadetud abipalve . Selle kirjutasid kaheksa Noarootsi lehmapidajat , kel piimaraha seitse kuud saamata . Kirjas seisab : " Võimaluse piires palume tasuta abi juristi või advokaadi näol , kes tunneks seadusi ja oskaks meid , vaeseid piimatootjaid , kaitsta . " Mida kuude kaupa saamata piimaraha tähendab , pole vaja seletada maainimesele , kellele see on üks põhilisi elatusallikaid . Mis kõige hullem - need inimesed on oma murega sisuliselt üksi , sest need , kes peaksid abi pakkuma , leiavad alati tuhat põhjendust , miks aidata pole võimalik . Põllumajandusministeerium püüab lahendusi otsida piimasektorile kiiranalüüsi tehes . Tekib küsimus , kas ülevaadet piimanduses toimuvast siis ministeeriumil varem polnudki . Kas senistest probleemidest pole õpitud ? Ja kas see hinnang tuleb ikka objektiivne või püütakse seisu euroläbirääkimiste taustal ilustada . Teised piimatööstused vaevalt Dairyt ja tema eelkäijat Lactot , kes kunagi sundis neid kokkuostuhinda kergitama , taga nutavad . Üks konkurent vähem . Pank muretseb jälle , kuidas oma laenu tagasi saada . Peaminister leiab , et rahaline abi raskustes firmadele on raiskamine . Aga maal oodatakse ja oodatakse piimaraha . Kuigi praegu töötatakse välja skeemi Dairy võlgade katmiseks , ei saa Dairyt vaadata ainult üksikjuhtumina . Kogu süsteem lonkab , ainult ei jõuta kokkuleppele , kumb jalg on haige või on tegu hoopis psüühikahäirega . Teistes , arenenumates riikides , kuuluvad piimatööstused enamasti ühistutele , mis on tootjate endi omad . Meil kunagi õitsval järjel olnud ühistegevust pole aga riik piisavalt toetada tahtnud . Et suuremad ärid õitseksid ja raha liiga laiali ei pudeneks . Ühe või paari lehma pidamine tundub selle kõrval ju nii tühine . Ent asi pole mitte lehmades , vaid inimestes . Kui asja kas või puhtmajanduslikult võtta , siis kumb on otstarbekam : kas maksta vaesematele maainimestele näljapajukit sotsiaaltoetuste näol või anda neile võimalus ise toime tulla . Läbi töö , mida nad oskavad ja austavad . Ka ilmset ebaõiglust võetakse tänases Eestis järjest enam loomuliku nähtusena . Nii ei jäägi talumehel ühel heal päeval muud üle , kui viimane lehm käekõrvale võtta ja advokaadi ukse taha viia . Piimaraha saamata , tapamajas järjekord , maksan natuuras .')

## Mapping test

mapping key on (sentence_nr, word_nr)  
mapping value on (sõna_start, sõna_end)

sõna_start väärtusega saab leida Text.words hulgast sobiva sõna ja selle indeksi


In [ ]:
def get_word_index_starting_at(EstNLTKText, start, end):
  word_indexes = []
  index = 0
  for word in EstNLTKText.words:
    if word.start <= start <= word.end or start <= word.start <= end:
      word_indexes.append(index)
      #return index
    index += 1
  return word_indexes

def sentnr_wordnr_to_wordID_in_words(EstNLTKText, sentnr, wordnr):
  startend = mapping.get((sentnr, wordnr))
  #print("start:", startend[0], "end", startend[1])
  return get_word_index_starting_at(EstNLTKText, startend[0], startend[1])


sentnr = 1
word = 3
print("Text.sentences.words ei anna õiget sõna kuna teisiti sõnestatud:", article_text.sentences[sentnr].words[word])
startend = mapping.get((sentnr, word))
print("Võib leida mitu vastet kuna .words võib näiteks '11-17' teha kolmeks osaks.")
#for i in get_word_index_starting_at(article_text, startend[0], startend[1]):
#  print("Sõna ideksi kaudu Text.words'ist:", article_text.words[i])
# peab andma 19. XI 1997

print()
# Annan https://raw.githubusercontent.com/soras/EstTimeMLCorpus/master/corpus/base-segmentation-morph-syntax järgi sentence_nr ja word_nr. Meetod tagastab indeksi, millega
# saab Text.words listist sõna kätte.
#print("Sõna indeksi kaudu wordist:", article_text.words[sentnr_wordnr_to_wordID_in_words(article_text, sentnr, word)])
#article_text.words

Text.sentences.words ei anna õiget sõna kuna teisiti sõnestatud: Span(',', [{'normalized_form': None}])
Võib leida mitu vastet kuna .words võib näiteks '11-17' teha kolmeks osaks.



## Automaatne TIMEXES kiht
See kiht vahest sisaldab õnneks ka ajaväljendeid, mida korpuses tähistatud pole ehk ajaväljendite arv kasvab.

In [ ]:
article_text.tag_layer(['timexes'] , resolver=TIMEXES_RESOLVER)
article_text.timexes
#article_text.gold_timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'tid': 't1', 'type': 'DATE', 'value': '2002-11-20', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['aastaid'], [{'tid': 't2', 'type': 'DURATION', 'value': 'PXY', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['kuude'], [{'tid': 't3', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['seitse', 'kuud'], [{'tid': 't4', 'type': 'DURATION', 'value': 'P7M', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['kuude'], [{'tid': 't5', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['praegu'], [{'tid': 't6', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['tänases'], [{'tid': 't7', 'type': 'DATE', 'value': '2002-11-21', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'tid': 't8', 'type': 'DATE', 'value': 'XXXX-XX-XX', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}])])

In [ ]:
article_text.gold_timexes # sentence_ID ja word_ID_in_sentence vastavad korpuse sõnestusele

Layer(name='gold_timexes', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[Span('20.11.2002', [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
Span('aastaid', [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
Span('kuude', [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('kaupa', [{'sentence_ID': 3, 'word_ID_in_sentence': 8, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('seitse', [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
Span('kuud', [{'sentence_ID': 5, 'word_ID_in_sentence': 9, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
Span('kuude', [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('kaupa', [{'sentence_ID': 7, 'word_ID_in_sentence': 2, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('varem', [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('senistest', [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('kunagi', [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('praegu', [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('kunagi', [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('tänases', [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('ühel', [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}]),
Span('heal', [{'sentence_ID': 28, 'word_ID_in_sentence': 5, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}]),
Span('päeval', [{'sentence_ID': 28, 'word_ID_in_sentence': 6, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

## IOB formaadis gold_word_events

In [ ]:
article_text.gold_word_events

Layer(name='gold_word_events', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilmekas', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('lihtsuses', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('kok

## Text.gold_timexes fraasi kaupa
Hetkel vanad meetodid ka siin. Kõige alumine( *fixed_gold_timexes_fraasi_kaupa( tekst )* ) annab kõige õigema tulemuse.  
Grupeerimise funktsiooni saaks ka vb mõnes teises meetodis kasutada, et kood oleks loetavam. + kommentaare eemaldada

In [ ]:
def lisa_gold_timexes_fraasi_kaupa(text_object):
  layer = estnltk.Layer(name='gold_timexes_phrases',
              text_object=text_object,
              attributes=['sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'timex_annotation', 'type', 'value'],
              enveloping='words')
  
  esimene_span = None
  eelmine_span = None
  type = None
  value = None
  phrase_spans = []
  usedWordIDs = []
  for span in text_object.gold_timexes:
    #print(span)
    #print("he", len(span.annotations))
    if not eelmine_span:
      phrase_spans = [span.base_span]
      esimene_span = span
      eelmine_span = span
      continue
    #print(span.timex_ID, eelmine_span.timex_ID)
    if span.timex_ID != eelmine_span.timex_ID and phrase_spans:
      #lisa annotation
      base_span = estnltk.EnvelopingBaseSpan(phrase_spans)
      new_span = estnltk.EnvelopingSpan(base_span, layer=layer)
      new_span.add_annotation(estnltk.Annotation(new_span, sentence_ID=esimene_span.sentence_ID, word_ID_in_sentence=esimene_span.word_ID_in_sentence, timex_ID=esimene_span.timex_ID, timex_annotation=esimene_span.timex_annotation, type=esimene_span.type, value=esimene_span.value))
      layer.add_span(new_span)
      phrase_spans = [span.base_span]
      esimene_span = span
    else:
      #kuulub eelmisega
      phrase_spans.append(span.base_span)

    eelmine_span = span

  # Viimase fraasi lisamine
  if phrase_spans:
    base_span = estnltk.EnvelopingBaseSpan(phrase_spans)
    new_span = estnltk.EnvelopingSpan(base_span, layer=layer)
    new_span.add_annotation(estnltk.Annotation(new_span, sentence_ID=esimene_span.sentence_ID[0], word_ID_in_sentence=esimene_span.word_ID_in_sentence[0], timex_ID=esimene_span.timex_ID[0], timex_annotation=esimene_span.timex_annotation, type=esimene_span.type, value=esimene_span.value))
    layer.add_span(new_span)

  text_object.add_layer(layer)
  
  return text_object

#lisa_gold_timexes_fraasi_kaupa(article_text)

def lisa_gold_timexes_fraasi_kaupa_ambiguous_gold_timexes(text_object):
  layer = estnltk.Layer(name='gold_timexes_phrases_ambiguous',
              text_object=text_object,
              attributes=['sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'],
              enveloping='words',
              ambiguous=True)
  
  esimene_span = None
  eelmine_span = None
  type = None
  value = None
  phrase_spans = []
  usedWordIDs = []
  for span in text_object.gold_timexes:
    print("span_anno", span.annotations)
    for annotation in span.annotations:
      #print(annotation.sentence_ID)
      #print("hehee", span.annotations[0])
      if not eelmine_span:
        phrase_spans = [span.base_span]
        esimene_span = span
        eelmine_span = span
        continue
      #print(span.timex_ID, eelmine_span.timex_ID)
      print("span.timex_ID", span.timex_ID, eelmine_span.timex_ID)
      # annotation.timex_ID[0] in
      # span.timex_ID !=
      #print(annotation.timex_ID, phrase_spans)
      if span.timex_ID != eelmine_span.timex_ID and phrase_spans:
        #lisa annotation
        print(annotation.timex_ID)
        base_span = estnltk.EnvelopingBaseSpan(phrase_spans)
        new_span = estnltk.EnvelopingSpan(base_span, layer=layer)
        for i in range(len(esimene_span.annotations)):
          print(new_span)
          new_span.add_annotation(estnltk.Annotation(new_span, sentence_ID=esimene_span.annotations[i].sentence_ID, word_ID_in_sentence=esimene_span.annotations[i].word_ID_in_sentence, timex_ID=esimene_span.annotations[i].timex_ID, timex_annotation=esimene_span.annotations[i].timex_annotation, type=esimene_span.annotations[i].type, value=esimene_span.annotations[i].value, expression=esimene_span.annotations[i].expression))
        layer.add_span(new_span)
        #print("lisan",new_span)
        phrase_spans = [span.base_span]
        esimene_span = span
      else:
        #kuulub eelmisega
        #lst.append(3) if 3 not in lst else lst
        phrase_spans.append(span.base_span) if span.base_span not in phrase_spans else phrase_spans
        #phrase_spans.append(span.base_span)

      eelmine_span = span

  # Viimase fraasi lisamine
  if phrase_spans:
    base_span = estnltk.EnvelopingBaseSpan(phrase_spans)
    new_span = estnltk.EnvelopingSpan(base_span, layer=layer)
    new_span.add_annotation(estnltk.Annotation(new_span, sentence_ID=esimene_span.sentence_ID[0], word_ID_in_sentence=esimene_span.word_ID_in_sentence[0], timex_ID=esimene_span.timex_ID[0], timex_annotation=esimene_span.timex_annotation[0], type=esimene_span.type[0], value=esimene_span.value[0], expression=esimene_span.expression[0]))
    layer.add_span(new_span)

  text_object.add_layer(layer)
  
  return text_object

def fixed_gold_timexes_fraasi_kaupa(text_objekt):
  layer = estnltk.Layer(name='gold_timexes_phrases_ambiguous_fixed',
              text_object=text_objekt,
              attributes=['sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'],
              enveloping='words',
              ambiguous=True)
  
  groups = text_objekt.gold_timexes.groupby(['timex_ID'], return_type='spans')
  phrase_spans = []
  for key in groups.count.keys():
    #print("key:",key)
    loendur = 0
    mitmes = 1000
    for annotationList in groups.groups[key]:
      for annotation in annotationList.annotations:
        #print(annotation)
        #print(annotation.timex_ID, key)
        if annotation.timex_ID in key:
          #print("lisan", annotation.timex_ID)
          phrase_spans.append(annotationList.base_span)
          if mitmes == 1000:
            mitmes = loendur
        loendur += 1
    base_span = estnltk.EnvelopingBaseSpan(phrase_spans)
    new_span = estnltk.EnvelopingSpan(base_span, layer=layer)
    #print(mitmes)
    new_span.add_annotation(estnltk.Annotation(new_span, sentence_ID=groups.groups[key][0].sentence_ID[mitmes], word_ID_in_sentence=groups.groups[key][0].word_ID_in_sentence[mitmes], timex_ID=groups.groups[key][0].timex_ID[mitmes], timex_annotation=groups.groups[key][0].timex_annotation[mitmes], type=groups.groups[key][0].type[mitmes], value=groups.groups[key][0].value[mitmes], expression=groups.groups[key][0].expression[mitmes]))
    #print("Lisada", new_span)
    try:
      layer.add_span(new_span)
    except:
      print("Selline span on juba olemas. Lisan veel ühe annotationi juurde.")
      layer.add_annotation(new_span, sentence_ID=groups.groups[key][0].sentence_ID[mitmes], word_ID_in_sentence=groups.groups[key][0].word_ID_in_sentence[mitmes], timex_ID=groups.groups[key][0].timex_ID[mitmes], timex_annotation=groups.groups[key][0].timex_annotation[mitmes], type=groups.groups[key][0].type[mitmes], value=groups.groups[key][0].value[mitmes], expression=groups.groups[key][0].expression[mitmes])
    phrase_spans = []
  text_objekt.add_layer(layer)
    #print(span)
#
#lisa_gold_timexes_fraasi_kaupa_ambiguous_gold_timexes(article_text)
fixed_gold_timexes_fraasi_kaupa(article_text)
article_text.gold_timexes_phrases_ambiguous_fixed

Layer(name='gold_timexes_phrases_ambiguous_fixed', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
EnvelopingSpan(['aastaid'], [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['varem'], [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['senistest'], [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['praegu'], [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['tänases'], [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

In [ ]:
article_text.gold_timexes_phrases_ambiguous_fixed

Layer(name='gold_timexes_phrases_ambiguous_fixed', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
EnvelopingSpan(['aastaid'], [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['varem'], [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['senistest'], [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['praegu'], [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['tänases'], [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

In [ ]:
article_text.timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'tid': 't1', 'type': 'DATE', 'value': '2002-11-20', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['aastaid'], [{'tid': 't2', 'type': 'DURATION', 'value': 'PXY', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['kuude'], [{'tid': 't3', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['seitse', 'kuud'], [{'tid': 't4', 'type': 'DURATION', 'value': 'P7M', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['kuude'], [{'tid': 't5', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['praegu'], [{'tid': 't6', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['tänases'], [{'tid': 't7', 'type': 'DATE', 'value': '2002-11-21', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'tid': 't8', 'type': 'DATE', 'value': 'XXXX-XX-XX', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None}])])

## Text.gold_timexes indeksite kandmine timexes kihti


In [ ]:
# word.start järgi
# See meetod annab timex_ID listist esimese väärtuse
def kanna_timex_ID_automaatsesse_kihti(tekstobjekt):
  tekstobjekt.timexes.attributes = tekstobjekt.timexes.attributes + ('timex_ID', )

  found = False
  for timex in tekstobjekt.timexes:
    found = False
    for gold_timex in tekstobjekt.gold_timexes_phrases_ambiguous:
      if timex.start == gold_timex.start:
        timex.timex_ID = gold_timex.timex_ID[0]
        found = True
    if not found:
      timex.timex_ID = "None"

# Olenevalt sellest, kas ainult word.start või nii .start kui ka .end peavad kattuma, kannab üle erineva arvu timex_ID'si.
# Muudab kihi mitmeseks ja lisab vajadusel samale span'ile mitu kirjet
def kanna_timex_ID_automaatsesse_kihti_ambiguous(tekstobjekt):
  matches = 0
  markimata = 0

  tekstobjekt.timexes.attributes = tekstobjekt.timexes.attributes + ('timex_ID', )
  tekstobjekt.timexes.ambiguous = True

  gold_IDs = set()
  found = False
  for timex in tekstobjekt.timexes:
    found = False
    miinimum_erinevus = 1000
    for gold_timex in tekstobjekt.gold_timexes_phrases_ambiguous_fixed:
      #if timex.start == gold_timex.start or timex.end == gold_timex.end: #  timex.start == gold_timex.start and timex.end == gold_timex.end
      if abs(timex.start - gold_timex.start) <= 15 and abs(timex.end - gold_timex.end) <= 15 or timex.start == gold_timex.start or timex.end == gold_timex.end: #  timex.start == gold_timex.start and timex.end == gold_timex.end
      #if timex.end == gold_timex.end:
        erinevus = abs(timex.start - gold_timex.start) + abs(timex.end - gold_timex.end)
        if erinevus <= miinimum_erinevus:
          matches += len(gold_timex.timex_ID)
          if len(gold_timex.timex_ID) == 1:
            timex.timex_ID = gold_timex.timex_ID[0]
            gold_IDs.add(gold_timex.timex_ID[0])
          else:
            timex.timex_ID = gold_timex.timex_ID[0]
            for i in range(1, len(gold_timex.timex_ID)):
              tekstobjekt.timexes.add_annotation(timex.base_span, tid=timex.tid[0], type=timex.type[0], value=timex.value[0], temporal_function=timex.temporal_function[0], anchor_time_id=timex.anchor_time_id[0], mod=timex.mod[0], quant=timex.quant[0], freq=timex.freq[0], begin_point=timex.begin_point[0], end_point=timex.end_point[0], part_of_interval=timex.part_of_interval[0], timex_ID=gold_timex.timex_ID[i])
              gold_IDs.add(gold_timex.timex_ID[i])
          found = True
          miinimum_erinevus = erinevus
    if not found:
      markimata += 1
      timex.timex_ID = "None"
      ##print("timex_ID puudu", timex)
  
  groups = tekstobjekt.timexes.groupby(['timex_ID'], return_type='spans')
  ##if len(groups.groups[('None',)]) > 0:
    ##print(tekstobjekt.gold_timexes_phrases_ambiguous_fixed)
    ##print()

  ##print("märkimata", markimata)
  return (len(gold_IDs), len(tekstobjekt.gold_timexes_phrases_ambiguous_fixed), markimata, len(tekstobjekt.timexes))
  #print("timex_ID väärtuseid üle kantud", len(gold_IDs))

result = kanna_timex_ID_automaatsesse_kihti_ambiguous(article_text)
article_text.timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval', 'timex_ID'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'tid': 't1', 'type': 'DATE', 'value': '2002-11-20', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't1'}]),
EnvelopingSpan(['aastaid'], [{'tid': 't2', 'type': 'DURATION', 'value': 'PXY', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't2'}]),
EnvelopingSpan(['kuude'], [{'tid': 't3', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't3'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'tid': 't4', 'type': 'DURATION', 'value': 'P7M', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't4'}]),
EnvelopingSpan(['kuude'], [{'tid': 't5', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't5'}]),
EnvelopingSpan(['praegu'], [{'tid': 't6', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't9'}]),
EnvelopingSpan(['tänases'], [{'tid': 't7', 'type': 'DATE', 'value': '2002-11-21', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't11'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'tid': 't8', 'type': 'DATE', 'value': 'XXXX-XX-XX', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't12'}])])

In [ ]:
article_text.gold_timexes_phrases_ambiguous_fixed

Layer(name='gold_timexes_phrases_ambiguous_fixed', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
EnvelopingSpan(['aastaid'], [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['varem'], [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['senistest'], [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['praegu'], [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['tänases'], [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

## Millised gold_timexes ajaväljendid pole timexes üle kantud?
puuduvaddict - filename ja gtimex.text elemendid juhul kui leidub gtimex.timex_IDsi mida timexes kihis pole  
pole_antud_vaartust - filename ja gtimex.text elemendid juhul kui kehtib 'puuduvaddict' JA timexes kihis on timex_ID väärtusega "None" 

In [ ]:
def puuduvad_gold_timexes(artikklid):
  puuduvaddict = dict()
  pole_antud_vaartust = dict()
  for artikkel in artikklid:
      puuduvad = []
      kantud = []
      for autoTimex in artikkel.timexes:
          kantud.append(autoTimex.timex_ID)
      for gtimex in artikkel.gold_timexes_phrases_ambiguous_fixed:
          if gtimex.timex_ID not in kantud:
              puuduvad.append(gtimex.text)
      if puuduvad:
          puuduvaddict[artikkel.meta.get("filename") + ".json"] = puuduvad
          onNone = ""
          for autoTimex in artikkel.timexes:
            #print(autoTimex.timex_ID)
            if autoTimex.timex_ID[0] == "None":
              pole_antud_vaartust[artikkel.meta.get("filename") + ".json"] = puuduvad
  return puuduvaddict, pole_antud_vaartust

puuduvad, pole_antud_vaartust = puuduvad_gold_timexes([article_text])
pole_antud_vaartust

{}

## Text objekti salvestamine json'ina

In [ ]:
def save_to_json(text_object):
  text_json = text_to_json(text_object)
  with open(drive_path + "artikklid/" + text_object.meta['filename'] + '.json', 'w') as fp:
      #pickle.dump(text_json, fp)
      json.dump(text_json, fp)
  print("Salvestatud", drive_path + "artikklid/" + text_object.meta['filename'] + '.json')

def text_from_json(filename):
  data = None
  with open(drive_path + "artikklid/" + filename, 'r') as fp:
      data = json.load(fp)
  return json_to_text(data)

# Mappingu salvestamine ja laadimine
def save_mapping(map, filename):
  with open(drive_path + "mapping/" + filename, 'wb') as outfile:
      pickle.dump(map, outfile)

def load_mapping(filename):
  with open(drive_path + "mapping/" + filename, 'rb') as infile:
      data = pickle.load(infile)
  return data

## Relatsioonide jaoks json faili loomine
Loon ühe json faili, mida saaks kiiremini sisse lugeda kui tekstifailidest ning on kergem päringuid teha.  
Sõnastiku struktuur on järgnev.   
 *Filename  
-------- main_events  
---------------- entity_id : values  
-------- sub_events  
---------------- entity_id : values  
-------- event_timex  
---------------- entity_id : values  
-------- event_dct  
---------------- entity_id : values  
 Filename ...*

Võimalik kood suhete kätte saamiseks.
relations.get(`<faili_nimi>`).get(`<main_events>`/`<sub_events>`/`<event_timex>`/`<event_dct>`).get(`<entity_ID>`)

---

### relations_v2
relations_v2 key: (filename, entity_id)
           value: selles failis entity_id'ga seotud relatsioonid


In [14]:
def esimene_viis():
  main_events_relations = load_relation_annotation(tlinkMainEventsFile)
  subordinate_events_relations = load_relation_annotation(tlinkSubEventsFile)
  event_timex_relations = load_relation_annotation(tlinkEventTimexFile)
  event_dct_relations = load_relation_to_dct_annotations(tlinkEventDCTFile)

  # Filename -> main_events, sub_events, event_timex, event_dct -> entity_id -> values
  # Filename
  #          main_events
  #                     entity_id : values
  #          sub_events
  #                     entity_id : values
  #          event_timex
  #                     entity_id : values
  #          event_dct
  #                     entity_id : values
  # Filename ...

  relations = {filename: {'main_events':main_events_relations.get(filename), 'sub_events':subordinate_events_relations.get(filename), 'event_timex':event_timex_relations.get(filename), 'event_dct':event_dct_relations.get(filename)}}

  print()
  print(relations)
  print(relations.get(filename))
  print(relations.get(filename).get('event_timex'))

def salvesta_relatsioonid(relatsioonid):
  with open(drive_path + 'relations.json', 'wb') as fp:
      #json.dump(relatsioonid, fp)
      pickle.dump(relatsioonid, fp)

def loe_relatsioonid():
  data = None
  with open(drive_path + 'relations.json', 'rb') as fp:
      #data = json.load(fp)
      data = pickle.load(fp)
  return data

#print()
#print(data)
#print(main_events_relations.get(filename))


def loo_relatsioonide_dict():
  main_events_relations = load_relation_annotation(tlinkMainEventsFile)
  subordinate_events_relations = load_relation_annotation(tlinkSubEventsFile)
  event_timex_relations = load_relation_annotation(tlinkEventTimexFile)
  event_dct_relations = load_relation_to_dct_annotations(tlinkEventDCTFile)

  # key = (filename, entity_id)
  # value = relations
  relations_v2 = {}
  for rels in [main_events_relations, subordinate_events_relations, event_timex_relations, event_dct_relations]:
    for text in rels:
      for entity_id in rels.get(text):
        if (text, entity_id) not in relations_v2:
          relations_v2[(text, entity_id)] = rels.get(text).get(entity_id)
        else:
          relations_v2.get((text, entity_id)).extend(rels.get(text).get(entity_id))

  return relations_v2

def only_main_events_relations():
  main_events_relations = load_relation_annotation(tlinkMainEventsFile)
  relations = {}
  for text in main_events_relations:
    for entity_id in main_events_relations.get(text):
      if (text, entity_id) not in relations:
        relations[(text, entity_id)] = main_events_relations.get(text).get(entity_id)
      else:
        relations.get((text, entity_id)).extend(main_events_relations.get(text).get(entity_id))
  return relations

def only_subordinate_events_relations():
  subordinate_events_relations = load_relation_annotation(tlinkSubEventsFile)
  relations = {}
  for text in subordinate_events_relations:
    for entity_id in subordinate_events_relations.get(text):
      if (text, entity_id) not in relations:
        relations[(text, entity_id)] = subordinate_events_relations.get(text).get(entity_id)
      else:
        relations.get((text, entity_id)).extend(subordinate_events_relations.get(text).get(entity_id))
  return relations


only_main_relatsioonid = only_main_events_relations()
#with open(drive_path + 'relations_only_main.json', 'wb') as fp:
#      #json.dump(relatsioonid, fp)
#      pickle.dump(only_main_relatsioonid, fp)

only_sub_relatsioonid = only_subordinate_events_relations()

#with open(drive_path + 'relations_only_sub.json', 'wb') as fp:
#      #json.dump(relatsioonid, fp)
#      pickle.dump(only_sub_relatsioonid, fp)

#print("\n Filename ja entity_id andmisel saab kätte kõik nendega seotud relatsioonid.")
#relations_v2.get((filename, 'e5'))
#print(main_events_relations.get(filename).get('e1'))
#print(subordinate_events_relations.get(filename).get('e1'))

{'e52', 'e61', 'e104', 'e71', 'e9', 'e34', 'e99', 'e93', 'e106', 'e94', 'e130', 'e120', 'e110', 'e118', 'e92', 'e30', 'e26', 'e89', 'e42', 'e72', 'e49', 'e103', 'e125', 'e131', 'e19', 'e17', 'e43', 'e135', 'e81', 'e108', 'e105', 'e134', 'e85', 'e29', 'e65', 'e10', 'e15', 'e50', 'e114', 'e82', 'e123', 'e76', 'e80', 'e79', 'e70', 'e4', 'e33', 'e51', 'e25', 'e18', 'e12', 'e35', 'e101', 'e44', 'e119', 'e128', 'e23', 'e133', 'e96', 'e54', 'e16', 'e3', 'e67', 'e63', 'e57', 'e14', 'e32', 'e7', 'e86', 'e77', 'e102', 'e115', 'e121', 'e109', 'e28', 'e137', 'e140', 'e75', 'e36', 'e98', 'e1', 'e13', 'e64', 'e126', 'e107', 'e129', 'e53', 'e88', 'e112', 'e58', 'e24', 'e124', 'e48', 'e90', 'e46', 'e95', 'e62', 'e56', 'e11', 'e22', 'e45', 'e60', 'e127', 'e84', 'e111', 'e20', 'e68', 'e74', 'e55', 'e38', 'e117', 'e83', 'e5', 'e73', 'e116', 'e78', 'e8', 'e100', 'e91', 'e113', 'e27', 'e21', 'e122', 'e87', 'e39', 'e47', 'e97', 'e66', 'e40', 'e59', 'e136', 'e139', 'e69', 'e31', 'e41', 'e37', 'e6', 'e2'}


# Terve korpuse peal jooksutamine

In [ ]:
baseAnnotations = load_base_segmentation(baseAnnotationFile)
(eventsByLoc, eventsByID) = load_entity_annotation(eventAnnotationFile)
articlesDCT = load_articles_DCT(articleMetadata_file)
(timexesByLoc, timexesByID) = load_entity_annotation(timexAnnotationFile)

In [ ]:
gold_timex_margenduste_arv = 0
ylekantud_timex_margendusi = 0
timexes_puudu = 0
timexes_arv = 0

# Iga artikkli puhul
for artikkel in articlesDCT.keys():
  print(artikkel)
  # Kaks layerit sündmuste (gold_events) ja ajamäärangute (*gold_timexes*) jaoks.
  # Text metaandmetes DCT.
  # ggold_word_events kiht (IOB formaadis märgendused)
  # (sentence_nr, word_nr) -> (start, end) -> word_id in 'words' mapping
  artikkli_tekst, mapping = create_Text_object(artikkel)
  # event_class "EVENT" -> õige class
  paranda_g_event_classid(artikkli_tekst)
  add_gold_word_events_w_classes(artikkli_tekst)
  add_gold_word_events_only_main(artikkli_tekst)

  # Text.words sõnade järgi
  add_gold_word_events_synced(artikkli_tekst)
  add_gold_word_events_w_classes_synced(artikkli_tekst)
  add_gold_word_events_only_main_synced(artikkli_tekst)
  save_mapping(mapping, artikkel + '.json')

  # Lisan automaatse TIMEXES kihi
  artikkli_tekst.tag_layer(['timexes'] , resolver=TIMEXES_RESOLVER)
  # gold_timexes kiht aga fraasi kaupa
  fixed_gold_timexes_fraasi_kaupa(artikkli_tekst)
  # gold_timexes_phrases kihi timex_ID'd kannan üle automaatsesse TIMEXES kihti
  result = kanna_timex_ID_automaatsesse_kihti_ambiguous(artikkli_tekst)
  gold_timex_margenduste_arv += result[1]
  ylekantud_timex_margendusi += result[0]
  timexes_puudu += result[2]
  timexes_arv += result[3]

  # Salvestan Text objekti JSON failina
  save_to_json(artikkli_tekst)

# Relatsioonide loomine ja salvestamine
relatsioonid = loo_relatsioonide_dict()
salvesta_relatsioonid(relatsioonid)

print()
print("Üle kanti", ylekantud_timex_margendusi, "gold_timexes märgendust", gold_timex_margenduste_arv,"-st.", "(Puudu jäi",  gold_timex_margenduste_arv - ylekantud_timex_margendusi, "märgendust 46 faili peale)")
print("timexes kihis jäi", timexes_puudu, "märgistusel", timexes_arv, "st lisamata. (34 märgistust 18 artikkli peale. Selliseid ajaväljendeid gold_timexes lihtsalt polnud)")
artikkli_tekst

aja_ml_2002_47.tasak.a006.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a006.sol.json
aja_ml_2002_47.tasak.a007.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a007.sol.json
aja_ml_2002_47.tasak.a008.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a008.sol.json
aja_ml_2002_47.tasak.a014.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a014.sol.json
aja_ml_2002_47.tasak.a016.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a016.sol.json
aja_ml_2002_47.tasak.a017.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a017.sol.json
aja_ml_2002_47.tasak.a023.sol
Salvestatud Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/artikklid/aja_ml_2002_47.tasak.a023.sol.json
aja_ml_2002_47.tasak.a025.sol
Salvestatud Loputo

Text(text='Kanadalast Piotr Staniaszeki tabas järjekordset mobiiliarvet avades ebameeldiv üllatus - mobiilifirma väitel oli ta kõnelnud rohkem kui 85 000 dollari ehk rohkem kui 890 930 krooni eest . Röögatu arve ei tekkinud siiski kallitest kaugekõnedest või lustiliinidele helistamisest . Staniaszek oli arvanud , et saab oma telefoni kasutada ka modemina ning mõnesajakroonise kuumaksu eest piiramatult netis surfata . Sidefirma Bell Mobility kundena laadis ta telefoni abil internetist alla täispikki mängufilme ja muid suuri faile . Et ta ületas nende allalaadimisega lepingus kindlaks määratud andmesidemahu , tiksutas telefonifirma arvet järjest suuremaks . Pärast seda , kui lepingutingimusi valesti mõistnud Staniaszek hiigelarve pärast lamenti lõi , tuli Bell Mobility talle vastu ning vähendas summa 3243 dollarile . Kuid Staniaszek pole ka sellega päri ja kavatseb arve vaidlustada . 22aastane Calgaryst pärit naftapuurija ütles novembrikuist arvet kommenteerides , et pidas seda esialgu eksituseks . Arve oli esmalt olnud 65 000 dollarit , piisavalt suur , ja mees võttis telefonifirmaga ühendust ning kurtis oma muret . Seks ajaks oli ta aga veel filme ja muud kraami alla laadinud ning arve oli kasvanud 85 000 dollarini . Staniaszeki sõnul on tema mobiiliarve tavaliselt umbes 150 dollarit ning seejuures saatvat Bell Mobility talle tihtipeale hoiatusi , et tema arve on väga suur . " Mõnikord on telefon välja lülitatud ka tavapärasest saja dollari võrra suurema arve puhul , " kurtis Staniaszek . " Nüüd on ülekulu 85 000 ja keegi ei võtnud vaevaks mind hoiatada . " Bell Mobility esindaja kinnitas , et firma on valmis arvet vähendama 3243 dollarile , sest just nii palju maksnuks Staniaszek allalaetud kraami eest juhul , kui ta kasutanuks kõige soodsamat andmesidelepingut .')

In [ ]:
artikkli_tekst.timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval', 'timex_ID'), spans=SL[EnvelopingSpan(['22aastane'], [{'tid': 't1', 'type': 'DURATION', 'value': 'P22Y', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't1'}]),
EnvelopingSpan(['novembrikuist'], [{'tid': 't2', 'type': 'DATE', 'value': '2007-11', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't2'}]),
EnvelopingSpan(['Nüüd'], [{'tid': 't3', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't3'}])])

## Vigade otsimine salvestatud tekstidest

In [ ]:
drive_path = "Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/"
artiklite_kaust = drive_path + "artikklid"

laetud_artiklid = []

with os.scandir(artiklite_kaust) as entries:
  for entry in entries:
    if entry.is_file():
      data = None
      with open(entry, 'r') as fp:
        data = json.load(fp)
      laetud_artiklid.append(json_to_text(data))

print("Lugesin sisse", len(laetud_artiklid), "artikklit.")

Lugesin sisse 80 artikklit.


In [ ]:
#uurida = list(articlesDCT.keys())[22]
# Märkasin nendes vigu, parandasin vastavalt, kontrollisin uuesti.
#aja_ml_2002_47.tasak.a016.sol
#aja_ml_2002_47.tasak.a038.sol
#aja_pm_2000_10_04.tasak.a015.sol
#aja_sloleht_2007_12_17.tasak.a016.sol
tekst = text_from_json("aja_ml_2002_47.tasak.a023.sol" + '.json')
tekst

Text(text='20.11.2002 Juba aastaid on piimavarumisel kestnud anarhia . Kui vahepeal tundubki , et suuremad hädad hakkavad mööda saama , tuleb uus löök . See , et Rapla Dairy on lehmapidajatele kuude kaupa võlgu , pälvis laiemat tähelepanu alles siis , kui riik tahtis firmat juhtivale Afganistani eruohvitserile Jaqub Haidaryle eriliste teenete eest kodakondsuse kinkida . Mure suuruse võtab oma ilmekas lihtsuses kokku põllumajandusministeeriumile saadetud abipalve . Selle kirjutasid kaheksa Noarootsi lehmapidajat , kel piimaraha seitse kuud saamata . Kirjas seisab : " Võimaluse piires palume tasuta abi juristi või advokaadi näol , kes tunneks seadusi ja oskaks meid , vaeseid piimatootjaid , kaitsta . " Mida kuude kaupa saamata piimaraha tähendab , pole vaja seletada maainimesele , kellele see on üks põhilisi elatusallikaid . Mis kõige hullem - need inimesed on oma murega sisuliselt üksi , sest need , kes peaksid abi pakkuma , leiavad alati tuhat põhjendust , miks aidata pole võimalik . Põllumajandusministeerium püüab lahendusi otsida piimasektorile kiiranalüüsi tehes . Tekib küsimus , kas ülevaadet piimanduses toimuvast siis ministeeriumil varem polnudki . Kas senistest probleemidest pole õpitud ? Ja kas see hinnang tuleb ikka objektiivne või püütakse seisu euroläbirääkimiste taustal ilustada . Teised piimatööstused vaevalt Dairyt ja tema eelkäijat Lactot , kes kunagi sundis neid kokkuostuhinda kergitama , taga nutavad . Üks konkurent vähem . Pank muretseb jälle , kuidas oma laenu tagasi saada . Peaminister leiab , et rahaline abi raskustes firmadele on raiskamine . Aga maal oodatakse ja oodatakse piimaraha . Kuigi praegu töötatakse välja skeemi Dairy võlgade katmiseks , ei saa Dairyt vaadata ainult üksikjuhtumina . Kogu süsteem lonkab , ainult ei jõuta kokkuleppele , kumb jalg on haige või on tegu hoopis psüühikahäirega . Teistes , arenenumates riikides , kuuluvad piimatööstused enamasti ühistutele , mis on tootjate endi omad . Meil kunagi õitsval järjel olnud ühistegevust pole aga riik piisavalt toetada tahtnud . Et suuremad ärid õitseksid ja raha liiga laiali ei pudeneks . Ühe või paari lehma pidamine tundub selle kõrval ju nii tühine . Ent asi pole mitte lehmades , vaid inimestes . Kui asja kas või puhtmajanduslikult võtta , siis kumb on otstarbekam : kas maksta vaesematele maainimestele näljapajukit sotsiaaltoetuste näol või anda neile võimalus ise toime tulla . Läbi töö , mida nad oskavad ja austavad . Ka ilmset ebaõiglust võetakse tänases Eestis järjest enam loomuliku nähtusena . Nii ei jäägi talumehel ühel heal päeval muud üle , kui viimane lehm käekõrvale võtta ja advokaadi ukse taha viia . Piimaraha saamata , tapamajas järjekord , maksan natuuras .')

### gold_events 5 esimest ja 5 viimast

In [ ]:
tekst.gold_events

Layer(name='gold_events', attributes=('sentence_ID', 'word_ID_in_sentence', 'event_ID', 'expression', 'event_annotation', 'event_class'), spans=SL[Span('piimavarumisel', [{'sentence_ID': 1, 'word_ID_in_sentence': 3, 'event_ID': 'e1', 'expression': '"piimavarumisel"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kestnud', [{'sentence_ID': 1, 'word_ID_in_sentence': 4, 'event_ID': 'e2', 'expression': '"kestnud"', 'event_annotation': 'EVENT ASPECTUAL comment="fraas: kestnud anarhia"', 'event_class': 'ASPECTUAL'}]),
Span('anarhia', [{'sentence_ID': 1, 'word_ID_in_sentence': 5, 'event_ID': 'e3', 'expression': '"anarhia"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tundubki', [{'sentence_ID': 2, 'word_ID_in_sentence': 2, 'event_ID': 'e4', 'expression': '"tundubki"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('hakkavad', [{'sentence_ID': 2, 'word_ID_in_sentence': 7, 'event_ID': 'e5', 'expression': '"hakkavad"', 'event_annotation': 'EVENT ASPECTUAL', 'event_class': 'ASPECTUAL'}]),
Span('saama', [{'sentence_ID': 2, 'word_ID_in_sentence': 9, 'event_ID': 'e6', 'expression': '"saama"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tuleb', [{'sentence_ID': 2, 'word_ID_in_sentence': 11, 'event_ID': 'e7', 'expression': '"tuleb"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('on', [{'sentence_ID': 3, 'word_ID_in_sentence': 5, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT STATE multiword="true"', 'event_class': 'STATE'}]),
Span('võlgu', [{'sentence_ID': 3, 'word_ID_in_sentence': 9, 'event_ID': 'e8', 'expression': '"on võlgu"', 'event_annotation': 'EVENT multiword="true"', 'event_class': 'STATE'}]),
Span('pälvis', [{'sentence_ID': 3, 'word_ID_in_sentence': 11, 'event_ID': 'e9', 'expression': '"pälvis"', 'event_annotation': 'EVENT OCCURRENCE comment="ka I_STATE: see, et on võlgu, pälvis tähelepanu"', 'event_class': 'OCCURRENCE'}]),
Span('tahtis', [{'sentence_ID': 3, 'word_ID_in_sentence': 19, 'event_ID': 'e10', 'expression': '"tahtis"', 'event_annotation': 'EVENT I_STATE', 'event_class': 'I_STATE'}]),
Span('teenete', [{'sentence_ID': 3, 'word_ID_in_sentence': 27, 'event_ID': 'e11', 'expression': '"teenete"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kinkida', [{'sentence_ID': 3, 'word_ID_in_sentence': 30, 'event_ID': 'e12', 'expression': '"kinkida"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('võtab', [{'sentence_ID': 4, 'word_ID_in_sentence': 2, 'event_ID': 'e13', 'expression': '"võtab"', 'event_annotation': 'EVENT STATE comment="fraas: võtab kokku"', 'event_class': 'STATE'}]),
Span('saadetud', [{'sentence_ID': 4, 'word_ID_in_sentence': 8, 'event_ID': 'e14', 'expression': '"saadetud"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('kirjutasid', [{'sentence_ID': 5, 'word_ID_in_sentence': 1, 'event_ID': 'e15', 'expression': '"kirjutasid"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('saamata', [{'sentence_ID': 5, 'word_ID_in_sentence': 10, 'event_ID': 'e16', 'expression': '"saamata"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('seisab', [{'sentence_ID': 6, 'word_ID_in_sentence': 1, 'event_ID': 'e17', 'expression': '"seisab"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('palume', [{'sentence_ID': 6, 'word_ID_in_sentence': 6, 'event_ID': 'e18', 'expression': '"palume"', 'event_annotation': 'EVENT I_ACTION', 'event_class': 'I_ACTION'}]),
Span('abi', [{'sentence_ID': 6, 'word_ID_in_sentence': 8, 'event_ID': 'e19', 'expression': '"abi"', 'event_annotation': 'EVENT OCCURRENCE', 'event_class': 'OCCURRENCE'}]),
Span('tunneks', [{'sentence_ID': 6, 'word_ID_in_sentence': 15, 'event_ID': 'e20', 'expression': '"tunneks"', 'event_annotation': 'EVENT STATE', 'event_class': 'STATE'}]),
Span('oskaks', 

In [ ]:
tekst.gold_word_events
for gwe in tekst.gold_word_events:
  if gwe.nertag != 'O':
    print(gwe.text, gwe.nertag)

piimavarumisel B-EVENT
kestnud B-EVENT
anarhia B-EVENT
tundubki B-EVENT
hakkavad B-EVENT
saama B-EVENT
tuleb B-EVENT
on B-EVENT
võlgu I-EVENT
pälvis B-EVENT
tahtis B-EVENT
teenete B-EVENT
kinkida B-EVENT
võtab B-EVENT
saadetud B-EVENT
kirjutasid B-EVENT
saamata B-EVENT
seisab B-EVENT
palume B-EVENT
abi B-EVENT
tunneks B-EVENT
oskaks B-EVENT
kaitsta B-EVENT
saamata B-EVENT
tähendab B-EVENT
pole B-EVENT
vaja I-EVENT
seletada B-EVENT
on B-EVENT
on B-EVENT
üksi I-EVENT
peaksid B-EVENT
abi B-EVENT
pakkuma B-EVENT
leiavad B-EVENT
aidata B-EVENT
pole B-EVENT
võimalik I-EVENT
püüab B-EVENT
lahendusi B-EVENT
otsida B-EVENT
kiiranalüüsi B-EVENT
tehes I-EVENT
Tekib B-EVENT
ülevaadet B-EVENT
toimuvast B-EVENT
polnudki I-EVENT
probleemidest B-EVENT
õpitud B-EVENT
hinnang B-EVENT
tuleb B-EVENT
püütakse B-EVENT
euroläbirääkimiste B-EVENT
ilustada B-EVENT
sundis B-EVENT
kergitama B-EVENT
nutavad B-EVENT
muretseb B-EVENT
laenu B-EVENT
saada B-EVENT
leiab B-EVENT
abi B-EVENT
on B-EVENT
raiskamine I-EVEN

### gold_word_events

In [ ]:
n = 100
tekst.gold_word_events_synced

Layer(name='gold_word_events_synced', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilmekas', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('lihtsuses', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSp

In [ ]:
tekst.gold_word_events_w_classes

Layer(name='gold_word_events_w_classes', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
Envelopi

In [ ]:
#sentence_starts = [lause.start for lause in tekst.sentences]
#print(sentence_starts)
#for entry in tekst.gold_word_events_only_main:
#  if entry.start in sentence_starts:
#    print(entry.text)

tekst.gold_word_events_only_main

Layer(name='gold_word_events_only_main', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT_ASPECTUAL', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT_I_STATE', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT_OCCURRENCE', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT_STATE', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilme

### gold_timexes 5 esimest ja 5 viimast

In [ ]:
tekst.gold_timexes[:5]

Layer(name='gold_timexes', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[Span('20.11.2002', [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
Span('aastaid', [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
Span('kuude', [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('kaupa', [{'sentence_ID': 3, 'word_ID_in_sentence': 8, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('seitse', [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}])])

In [ ]:
tekst.gold_timexes

Layer(name='gold_timexes', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[Span('20.11.2002', [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
Span('aastaid', [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
Span('kuude', [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('kaupa', [{'sentence_ID': 3, 'word_ID_in_sentence': 8, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('seitse', [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
Span('kuud', [{'sentence_ID': 5, 'word_ID_in_sentence': 9, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
Span('kuude', [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('kaupa', [{'sentence_ID': 7, 'word_ID_in_sentence': 2, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
Span('varem', [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('senistest', [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('kunagi', [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('praegu', [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('kunagi', [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
Span('tänases', [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
Span('ühel', [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}]),
Span('heal', [{'sentence_ID': 28, 'word_ID_in_sentence': 5, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}]),
Span('päeval', [{'sentence_ID': 28, 'word_ID_in_sentence': 6, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

### gold_timexes_phrases

In [ ]:
tekst.gold_timexes_phrases_ambiguous_fixed

Layer(name='gold_timexes_phrases_ambiguous_fixed', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"20.11.2002"', 'timex_annotation': 'TIMEX DATE 2002-11-20 functionInDocument="CREATION_TIME"', 'type': 'DATE', 'value': '2002-11-20'}]),
EnvelopingSpan(['aastaid'], [{'sentence_ID': 1, 'word_ID_in_sentence': 1, 'timex_ID': 't2', 'expression': '"aastaid"', 'timex_annotation': 'TIMEX DURATION PXY', 'type': 'DURATION', 'value': 'PXY'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 3, 'word_ID_in_sentence': 7, 'timex_ID': 't3', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'sentence_ID': 5, 'word_ID_in_sentence': 8, 'timex_ID': 't4', 'expression': '"seitse kuud"', 'timex_annotation': 'TIMEX DURATION P7M multiword="true"', 'type': 'DURATION', 'value': 'P7M'}]),
EnvelopingSpan(['kuude', 'kaupa'], [{'sentence_ID': 7, 'word_ID_in_sentence': 1, 'timex_ID': 't5', 'expression': '"kuude kaupa"', 'timex_annotation': 'TIMEX DURATION PXM multiword="true"', 'type': 'DURATION', 'value': 'PXM'}]),
EnvelopingSpan(['varem'], [{'sentence_ID': 10, 'word_ID_in_sentence': 9, 'timex_ID': 't6', 'expression': '"varem"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['senistest'], [{'sentence_ID': 11, 'word_ID_in_sentence': 1, 'timex_ID': 't7', 'expression': '"senistest"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 13, 'word_ID_in_sentence': 10, 'timex_ID': 't8', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['praegu'], [{'sentence_ID': 18, 'word_ID_in_sentence': 1, 'timex_ID': 't9', 'expression': '"praegu"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['kunagi'], [{'sentence_ID': 21, 'word_ID_in_sentence': 1, 'timex_ID': 't10', 'expression': '"kunagi"', 'timex_annotation': 'TIMEX DATE PAST_REF', 'type': 'DATE', 'value': 'PAST_REF'}]),
EnvelopingSpan(['tänases'], [{'sentence_ID': 27, 'word_ID_in_sentence': 4, 'timex_ID': 't11', 'expression': '"tänases"', 'timex_annotation': 'TIMEX DATE PRESENT_REF', 'type': 'DATE', 'value': 'PRESENT_REF'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'sentence_ID': 28, 'word_ID_in_sentence': 4, 'timex_ID': 't12', 'expression': '"ühel heal päeval"', 'timex_annotation': 'TIMEX DATE XXXX-XX-XX multiword="true"', 'type': 'DATE', 'value': 'XXXX-XX-XX'}])])

### automaatne timexes

In [ ]:
tekst.timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval', 'timex_ID'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'tid': 't1', 'type': 'DATE', 'value': '2002-11-20', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't1'}]),
EnvelopingSpan(['aastaid'], [{'tid': 't2', 'type': 'DURATION', 'value': 'PXY', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't2'}]),
EnvelopingSpan(['kuude'], [{'tid': 't3', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't3'}]),
EnvelopingSpan(['seitse', 'kuud'], [{'tid': 't4', 'type': 'DURATION', 'value': 'P7M', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't4'}]),
EnvelopingSpan(['kuude'], [{'tid': 't5', 'type': 'DURATION', 'value': 'PXM', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't5'}]),
EnvelopingSpan(['praegu'], [{'tid': 't6', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't9'}]),
EnvelopingSpan(['tänases'], [{'tid': 't7', 'type': 'DATE', 'value': '2002-11-21', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't11'}]),
EnvelopingSpan(['ühel', 'heal', 'päeval'], [{'tid': 't8', 'type': 'DATE', 'value': 'XXXX-XX-XX', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't12'}])])

In [ ]:
tekst.gold_word_events_synced

Layer(name='gold_word_events_synced', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilmekas', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('lihtsuses', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSp

Väike kontroll gold_word_events_x ja synced vahel.  
Artikklis 'aja_pm_2000_10_04.tasak.a029.sol' on sõna "Party'lgi" .words kihis kolme osana ja seepärast erinevus.

In [ ]:
for artikkel in laetud_artiklid:
  tavaline = 0
  for tava in artikkel.gold_word_events:
    if tava.nertag != "O":
      tavaline += 1
  synced = 0
  for sync in artikkel.gold_word_events_synced:
    if sync.nertag != "O":
      synced += 1
  #print()
  if tavaline != synced:
    print("word_events", artikkel.meta.get("filename"))

  tavaline = 0
  for tava in artikkel.gold_word_events_w_classes:
    if tava.nertag != "O":
      tavaline += 1
  synced = 0
  for sync in artikkel.gold_word_events_w_classes_synced:
    if sync.nertag != "O":
      synced += 1
  #print()
  if tavaline != synced:
    print("word_events_w_classes", artikkel.meta.get("filename"))

  tavaline = 0
  for tava in artikkel.gold_word_events_only_main:
    if tava.nertag != "O":
      tavaline += 1
  synced = 0
  for sync in artikkel.gold_word_events_only_main_synced:
    if sync.nertag != "O":
      synced += 1
  #print()
  if tavaline != synced:
    print("word_events_only_main", artikkel.meta.get("filename"))

word_events aja_pm_2000_10_04.tasak.a029.sol
word_events_w_classes aja_pm_2000_10_04.tasak.a029.sol
word_events_only_main aja_pm_2000_10_04.tasak.a029.sol


### Puuduvad gold_timexes ajaväljendid timexes kihist
Vaatan artikkleid, milles jäi timexes kihis mingile väljendile timex_ID panemata.  
puuduvad - filename ja gtimex.text elemendid juhul kui leidub gtimex.timex_IDsi mida timexes kihis pole  
pole_antud_vaartust - filename ja gtimex.text elemendid juhul kui kehtib 'puuduvaddict' JA timexes kihis on timex_ID väärtusega "None" ehk võis ülekandmisel probleeme tekkida

In [ ]:
puuduvad, pole_antud_vaartust = puuduvad_gold_timexes(laetud_artiklid)
keyd = list(puuduvad.keys())
len(pole_antud_vaartust)

11

In [ ]:
keyd = list(pole_antud_vaartust.keys())
artikkel = keyd[10]
print(artikkel)
print(pole_antud_vaartust.get(artikkel))
tekst = text_from_json(artikkel)
tekst.gold_timexes_phrases_ambiguous_fixed

aja_sloleht_2007_12_17.tasak.a040.sol.json
[['Kolmapäevaõhtuses'], ['teist', 'aastat']]


Layer(name='gold_timexes_phrases_ambiguous_fixed', attributes=('sentence_ID', 'word_ID_in_sentence', 'timex_ID', 'expression', 'timex_annotation', 'type', 'value'), spans=SL[EnvelopingSpan(['Kolmapäevaõhtuses'], [{'sentence_ID': 0, 'word_ID_in_sentence': 0, 'timex_ID': 't1', 'expression': '"Kolmapäevaõhtuses"', 'timex_annotation': 'TIMEX TIME 2007-12-12TEV', 'type': 'TIME', 'value': '2007-12-12TEV'}]),
EnvelopingSpan(['kolmapäeva', 'õhtul'], [{'sentence_ID': 2, 'word_ID_in_sentence': 13, 'timex_ID': 't2', 'expression': '"kolmapäeva õhtul"', 'timex_annotation': 'TIMEX TIME 2007-12-12TEV multiword="true"', 'type': 'TIME', 'value': '2007-12-12TEV'}]),
EnvelopingSpan(['teist', 'aastat'], [{'sentence_ID': 3, 'word_ID_in_sentence': 8, 'timex_ID': 't3', 'expression': '"teist aastat"', 'timex_annotation': 'TIMEX DATE XXXX comment="semantika edasiandmine probleemne" multiword="true"', 'type': 'DATE', 'value': 'XXXX'}]),
EnvelopingSpan(['tund', 'aega'], [{'sentence_ID': 4, 'word_ID_in_sentence': 9, 'timex_ID': 't4', 'expression': '"tund aega"', 'timex_annotation': 'TIMEX DURATION PT1H multiword="true"', 'type': 'DURATION', 'value': 'PT1H'}]),
EnvelopingSpan(['nädala', 'keskpaigani'], [{'sentence_ID': 16, 'word_ID_in_sentence': 8, 'timex_ID': 't5', 'expression': '"nädala keskpaigani"', 'timex_annotation': 'TIMEX DATE 2007-W51 mod="MID" multiword="true"', 'type': 'DATE', 'value': '2007-W51'}]),
EnvelopingSpan(['Eile'], [{'sentence_ID': 17, 'word_ID_in_sentence': 0, 'timex_ID': 't6', 'expression': '"Eile"', 'timex_annotation': 'TIMEX DATE 2007-12-16', 'type': 'DATE', 'value': '2007-12-16'}]),
EnvelopingSpan(['homme'], [{'sentence_ID': 17, 'word_ID_in_sentence': 7, 'timex_ID': 't7', 'expression': '"homme"', 'timex_annotation': 'TIMEX DATE 2007-12-18', 'type': 'DATE', 'value': '2007-12-18'}]),
EnvelopingSpan(['kolmapäeval'], [{'sentence_ID': 17, 'word_ID_in_sentence': 14, 'timex_ID': 't8', 'expression': '"kolmapäeval"', 'timex_annotation': 'TIMEX DATE 2007-12-19', 'type': 'DATE', 'value': '2007-12-19'}])])

In [ ]:
tekst.timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval', 'timex_ID'), spans=SL[EnvelopingSpan(['kolmapäeva', 'õhtul'], [{'tid': 't1', 'type': 'TIME', 'value': '2007-12-19TEV', 'temporal_function': True, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't2'}]),
EnvelopingSpan(['tund', 'aega'], [{'tid': 't2', 'type': 'DURATION', 'value': 'PT1H', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't4'}]),
EnvelopingSpan(['17', 'tuntud'], [{'tid': 't3', 'type': 'DURATION', 'value': 'PT17H', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 'None'}]),
EnvelopingSpan(['nädala', 'keskpaigani'], [{'tid': 't4', 'type': 'DATE', 'value': 'XXXX-WXX', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': 'MID', 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't5'}]),
EnvelopingSpan(['Eile'], [{'tid': 't5', 'type': 'DATE', 'value': '2007-12-16', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't6'}]),
EnvelopingSpan(['homme'], [{'tid': 't6', 'type': 'DATE', 'value': '2007-12-18', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't7'}]),
EnvelopingSpan(['kolmapäeval'], [{'tid': 't7', 'type': 'DATE', 'value': '2007-12-19', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't8'}])])

# Treening- ja testhulk

## Imports

In [ ]:
from sklearn.model_selection import KFold
import os

## Loe artiklid sisse

In [ ]:
artiklite_kaust = drive_path + "artikklid"

laetud_artiklid = []

with os.scandir(artiklite_kaust) as entries:
  for entry in entries:
    if entry.is_file():
      data = None
      with open(entry, 'r') as fp:
        data = json.load(fp)
      laetud_artiklid.append(json_to_text(data))

print("Lugesin sisse", len(laetud_artiklid), "artikklit.")
print(type(laetud_artiklid[0]))

Lugesin sisse 80 artikklit.
<class 'estnltk.text.Text'>


In [ ]:
events = []
otsitav = None
for artikel in laetud_artiklid:
    events.append([event.nertag for event in artikel.gold_word_events])
    if artikel.meta.get("filename") == 'aja_ml_2002_47.tasak.a023.sol':
      otsitav = artikel
#print(events)
#laetud_artiklid[3].gold_word_events_only_main_synced
rels = loe_relatsioonid()
#otsitav.timexes
laetud_artiklid[5].timexes

Layer(name='timexes', attributes=('tid', 'type', 'value', 'temporal_function', 'anchor_time_id', 'mod', 'quant', 'freq', 'begin_point', 'end_point', 'part_of_interval', 'timex_ID'), spans=SL[EnvelopingSpan(['20.11.2002'], [{'tid': 't1', 'type': 'DATE', 'value': '2002-11-20', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't1'}]),
EnvelopingSpan(['Suvel'], [{'tid': 't2', 'type': 'DATE', 'value': '2002-SU', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't2'}]),
EnvelopingSpan(['nüüdsest'], [{'tid': 't3', 'type': 'DATE', 'value': 'PRESENT_REF', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't3'}]),
EnvelopingSpan(['Aastast', '1893'], [{'tid': 't4', 'type': 'DATE', 'value': '1893', 'temporal_function': False, 'anchor_time_id': None, 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': {'tid': 't6', 'type': 'DURATION', 'value': 'PXXM', 'temporal_function': True, 'begin_point': 't4', 'end_point': 't5'}, 'timex_ID': 't6'}]),
EnvelopingSpan(['tänavuse', 'suveni'], [{'tid': 't5', 'type': 'DATE', 'value': '2002-SU', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': {'tid': 't6', 'type': 'DURATION', 'value': 'PXXM', 'temporal_function': True, 'begin_point': 't4', 'end_point': 't5'}, 'timex_ID': 't7'}]),
EnvelopingSpan(['eelmisel', 'aastal'], [{'tid': 't7', 'type': 'DATE', 'value': '2001', 'temporal_function': True, 'anchor_time_id': 't0', 'mod': None, 'quant': None, 'freq': None, 'begin_point': None, 'end_point': None, 'part_of_interval': None, 'timex_ID': 't8'}])])

## Loo test ja treeninghulgad

In [ ]:
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(laetud_artiklid)
print(kf)
print()
print("4-ja alamhulgaga saadud artiklite indeksid.")
for train_index, test_index in kf.split(laetud_artiklid):
  print("TRAIN:", train_index)
  print("TEST:", test_index)
  print()

KFold(n_splits=10, random_state=None, shuffle=True)

4-ja alamhulgaga saadud artiklite indeksid.
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 22 23 24 26
 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47 48 49 51 52
 53 54 55 56 57 58 59 60 62 63 64 65 66 67 68 69 70 71 73 74 75 77 78 79]
TEST: [10 21 25 35 50 61 72 76]

TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 52 54 55 56 58 59 60 61 62 64 65 66 67 68 69 70 71 72 73 74 76 77 78]
TEST: [ 0 26 36 53 57 63 75 79]

TRAIN: [ 0  2  4  5  6  7  9 10 11 12 13 14 15 16 17 19 20 21 22 24 25 26 28 29
 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59 60 61 62 63 64 65 66 68 69 70 71 72 74 75 76 77 78 79]
TEST: [ 1  3  8 18 23 27 67 73]

TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25
 26 27 28 31 32 33 34 35 36 37 38 39 40 41 